In [ ]:
;pp# Only run the first time you open colab notebook
%pip install spotipy
%pip install requests 
%pip install pandas
%pip install unidecode

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 153kB 14.9MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
     |████████████████████████████████| 245kB 6.7MB/s 


Note: we did not import ClientCredentials from spotipy.oauth2 because only endpoints that do not access user information can be accessed using Client  Credentials Flow



In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth

import json
import pandas as pd
import io
  
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/MyDrive/Menon_Labs/code_and_data'

from unidecode import unidecode

Mounted at /content/gdrive


In [ ]:
#Our third-party app needs user permission to access their data 
#Provide redirect_uri that will be a redirect link used to collect the user's permisson 
# Spotify Developer dashboard > Edit Settings > Redirect URI > add a link (i.e. http:..localhost:8000/callback)
#Set this as our REDIRECT_URI

In [ ]:
# Query the API's endpoints 
#Change the Client ID and Client Secret to start querying the API's endpoints
  #Set these as environment variables or pass these into auth_manager 

USERNAME = '2fzeav2rzmdeyqjl8te6n23ce'
CLIENT_ID = 'f83ae0649a9149c18f987350686bc04c'
CLIENT_SECRET = 'f11446ead58f451d9a75e2b2a4247703'
REDIRECT_URI = 'http://localhost:8000/callback'



https://github.com/mtoto/mtoto.github.io/tree/master/about/about

https://towardsdatascience.com/get-your-spotify-streaming-history-with-python-d5a208bbcbd3




To train a ML model later on, we need more data so we used an existing JSON file created by using the Spotify API. This dataset has 3200+ observations (i.e. tracks) that were collected by making calls to the Spotify Web API every 3 hours, for 3 months.

Source: http://tamaszilagyi.com/blog/2017/2017-06-02-spotifyr/

## **Importing Data**

In [ ]:
#Upload the JSON file from local machine
uploaded = files.upload()


Saving user_tracks.json to user_tracks.json


In [ ]:
file_name = "user_tracks.json"

#Decode JSON file
io.StringIO(uploaded[file_name].decode("utf-8"))

#Parse JSON file into a Python dict
user_tracks_dict = json.loads(uploaded[file_name].decode("utf-8"))

#Check if uploaded correctly
user_tracks_dict[1]

{'artist_href': ['https://api.spotify.com/v1/artists/0Vd8YQz8TYk2vSKEYVvIgL'],
 'artist_id': ['0Vd8YQz8TYk2vSKEYVvIgL'],
 'artist_name': ['Dojo Cuts'],
 'artist_urls': ['https://open.spotify.com/artist/0Vd8YQz8TYk2vSKEYVvIgL'],
 'disc_number': 1,
 'duration_ms': 226426,
 'explicit': True,
 'href': 'https://api.spotify.com/v1/tracks/5DXq9lQxg6tF88XVzwOKGA',
 'played_at': '2017-06-24T18:52:23.008Z',
 'playlist_href': 'https://api.spotify.com/v1/users/1170891844/playlists/29XAftFCmwVBJ64ROX8gzA',
 'playlist_url': 'https://open.spotify.com/user/1170891844/playlist/29XAftFCmwVBJ64ROX8gzA',
 'preview_url': 'https://p.scdn.co/mp3-preview/0aa5a1de7689cd8333aa59500525fb40fb07f72e?cid=448b90c4e62f4624a56d9a25653afc8e',
 'track_id': '5DXq9lQxg6tF88XVzwOKGA',
 'track_name': 'I Can Give',
 'track_number': 2,
 'type': 'playlist',
 'uri': 'spotify:user:1170891844:playlist:29XAftFCmwVBJ64ROX8gzA'}

##**Data Cleaning and Preprocessing**

In [ ]:
#Get artist_name, artist_id, track_name, track_id and played_at from the dict and create a dataframe storing these variables

#init
artist_name = []
artist_id = []
track_name = []
track_id = []
played_at = []


for i,t in enumerate(user_tracks_dict):
  #Convert all the list elements to str 
  #Ensures we get a list of strs instead of a list of lists
  listToStr_artistName = ' '.join([str(elem) for elem in user_tracks_dict[i]['artist_name']])
  listToStr_artistID = ' '.join([str(elem) for elem in user_tracks_dict[i]['artist_id']])
  listToStr_trackName = ''.join([str(elem) for elem in user_tracks_dict[i]['track_name']])
  listToStr_trackID = ''.join([str(elem) for elem in user_tracks_dict[i]['track_id']])
  listToStr_playedAT = ''.join([str(elem) for elem in user_tracks_dict[i]['played_at']])

  #Add elements to the lists 
  artist_name.append(listToStr_artistName)
  artist_id.append(listToStr_artistID)
  track_name.append(listToStr_trackName)
  track_id.append(listToStr_trackID)
  played_at.append(listToStr_playedAT)


#create dataframe using lists

df_user_tracks = pd.DataFrame(list(zip(artist_name, artist_id, track_name, track_id, played_at)),
               columns =['Artist', 'Artist ID', 'Track', 'Track ID', 'Time Played At'])

df_user_tracks


,Artist,Artist ID,Track,Track ID,Time Played At
0,Michael Franti & Spearhead,1mHuZMOP8FG5ip4yAb1vrB,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z
1,Dojo Cuts,0Vd8YQz8TYk2vSKEYVvIgL,I Can Give,5DXq9lQxg6tF88XVzwOKGA,2017-06-24T18:52:23.008Z
2,The Young Punx Cagedbaby D-Tale,3gkX03yW2WrvcDMp04TKjN 2sbZiktyjJtAAWs3LHeHzO ...,Loving You Baby,1Y7u4vDBYUqGClwy917MUm,2017-05-03T12:42:42.156Z
3,Mage,0q7PuJHIkWuHiHezAQFB6u,All The Time,0t0zDIeQykxGQUV5UHDgNA,2017-06-10T09:29:39.470Z
4,Bird of Prey Iguana,2mT2d7wMYhVOrQrjXMRWYS 66wBC351mvRza9Iw8pPn0M,Xenomorphic,6riNnIsy4o3iIdIADdmcqO,2017-06-27T00:24:42.441Z
...,...,...,...,...,...
3269,Nicole Willis & the Soul Investigators,1qR7R3OBeHqN7Dl12SBUMl,Holdin' On,5XekVKJWtC1d6C4VyQyLrQ,2017-04-06T23:16:23.576Z
3270,Nick Monaco,0fsDVdEDqPmn5CISll6OIQ,The Stalker,1zE1lRl7Xrt5fidhKzAktU,2017-04-07T13:30:40.276Z
3271,Mr President,0vxQClb5BOpAPfYZl1l0fR,Love and happiness,3c8UDT5mKtn81PCzinsA9Y,2017-04-24T08:42:06.233Z
3272,Bomba Estéreo,5n9bMYfz9qss2VOW89EVs2,Bosque,6jFI3qezWZdTaCZktJDhf9,2017-04-25T12:59:21.212Z


Now, using terminal, we get the audio features of all the tracks listed above. First, we need to create a string of all the track IDs, separated by commas, so we can use it with the Spotify Web API tool online. This will generate the terminal command we need to extract the audio features of the above tracks. 


We then write that command to a separate json file that we can upload from our local machine.

In [ ]:
#GET audio features of all the tracks

#Create a string of all the track_ids above, separated by "," char 

track_id_str = ''

for id in df_user_tracks['Track ID']:
  track_id_str +=  id + ',' 

track_id_str

#Use string of track_ids to retrieve command reqd for terminal from Spotify Web API to extract audio features
#Modify command to write the output to a separate JSON file 

'1SZThDcsBdExLGgsGF4jwj,5DXq9lQxg6tF88XVzwOKGA,1Y7u4vDBYUqGClwy917MUm,0t0zDIeQykxGQUV5UHDgNA,6riNnIsy4o3iIdIADdmcqO,0yXij8RLCsp2Ir3Wkv9bCS,5s9wI2Jijxp5sDyL9ltKmR,2P5mABk27DkOT8Gi2NbTpO,7pN8GMLw3prWFn5nvJRM6O,5hGBJKU8aJaZPVSV7NxoTp,6mz1fBdKATx6qP4oP1I65G,5HOyaBTI0ypXjyowSRKeHj,1GIeXkjmmoYVFoQpbV6sXD,2PMZRVTJlDj2DokOxYnmnY,1NaRFCn2s3Rb75ghDIfBuR,61HCD3QERj7kxp4qHSzGiF,2Ds6mEvudpm8tlBY8s6pPl,15VvVzNAne1ndguY5Rh8ji,2uWoy80sBB8a59nh0cDSe1,4RY7JyzWaluUWKg3zPKOdk,5iQ6QNbeoIBSUuxLvyZL4Q,3qQWGXvNwIOrskKOK9u9A0,2p4cHvREkcBQT7IzqwliHi,1Mxx5BNUnUmVESTaPgblt2,7mwfuAhF0DclPHLhKEyYdl,7bVU2AF4JsrXzPCvrxXw7X,5DoHizYw9jyvyChGs04VIB,56SqXxIisOSzSrJXuLBMY8,3RCmcAmUn9YRnNNDXX4sgk,1uEZdfC78Cg5lxagSv9Dde,6pD5lJP9upILAGe9drWqXU,7c3gg0yaitZpzx3EbuD2ch,16Po3i2Kcsy74jz0lp6fdJ,31DcoNNQ5OR3yh0BVc13fA,5lGAa7HUT1JmNXC1uXcv6x,5rbOfVxNxWWW73zsu0knPa,6LHCmgP2PGmZCjPDijxh7r,75Fupwxut5dZfw7rgTi2wK,1IN17M6IaDuMsRtuU3CADn,2JR1WeETsXmFR1kRC2oXXq,6PKpNHp2O1oNrOK7ZLU0qq,7JwwAlWgTHpWkvRUbXbkVb,5DXq9lQxg6tF88XVzwOKGA,77AwEu7qGs

In [ ]:

#Upload audio_features JSON file from local machine
uploaded_af = files.upload()

Saving audio_features_alltracks.json to audio_features_alltracks.json


In [ ]:
file_name = "audio_features_alltracks.json"

#Decode JSON file
io.StringIO(uploaded_af[file_name].decode("utf-8"))

#Parse JSON file into a Python dict
audio_features_dict = json.loads(uploaded_af[file_name].decode("utf-8"))


In [ ]:
#Checking what type of keys and values there are 
keys, values = zip(*audio_features_dict.items())
keys
values

#There is one {key: value} pair in this dictionary, where the value is a tuple
type(values)

#Accessing audio features of a single track, track indexed by 0 
values[0]

# Audio features of each track is a list
type(values[0])

#Total 3256 tracks 
for value in audio_features_dict.values():
  num_of_tracks = len(value)

#Accessing each attribute for each trackID. For trackID at index 0, this is how we access its danceability
audio_features_dict['audio_features'][0]['danceability']

#Testing to see values
audio_features_dict['audio_features'][3255]['danceability']
type(audio_features_dict['audio_features'][0]['danceability'])

float

In [ ]:
#Get danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, track_id from the dict and create a dataframe storing these variables

#init
track_id_af = []
danceability = []
energy = []
loudness = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = [] 

#Iterate over the dictionary and create separate arrays for each feature 
for i in range(0, num_of_tracks):
  #Add elements to the lists 
  danceability.append(audio_features_dict['audio_features'][i]['danceability'])
  energy.append(audio_features_dict['audio_features'][i]['energy'])
  loudness.append(audio_features_dict['audio_features'][i]['loudness'])
  speechiness.append(audio_features_dict['audio_features'][i]['speechiness'])
  acousticness.append(audio_features_dict['audio_features'][i]['acousticness'])
  instrumentalness.append(audio_features_dict['audio_features'][i]['instrumentalness'])
  liveness.append(audio_features_dict['audio_features'][i]['liveness'])
  valence.append(audio_features_dict['audio_features'][i]['valence'])
  tempo.append(audio_features_dict['audio_features'][i]['tempo'])
  track_id_af.append(audio_features_dict['audio_features'][i]['id'])


  #Convert all the list elements to str 
  # #Ensures we get a list of strs instead of a list of lists
  # listToStr_Danceability = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['danceability']])
  # listToStr_Energy = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['energy']])
  # listToStr_Loudness = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['loudness']])
  # listToStr_Speechiness = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['speechiness']])
  # listToStr_Acousticness = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['acousticness']])
  # listToStr_Instrumentalness = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['instrumentalness']])
  # listToStr_Liveness = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['liveness']])
  # listToStr_Valence = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['valence']])
  # listToStr_Tempo = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['tempo']])
  # listToStr_trackID = ' '.join([str(elem) for elem in audio_features_dict['audio_features'][i]['id']])

  
#create dataframe using lists

df_audio_features = pd.DataFrame(list(zip(track_id_af, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo)),
               columns =['Track ID','Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo'])

# df_audio_features has 3256 rows and 10 cols 
df_audio_features.shape

df_audio_features


,Track ID,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,1SZThDcsBdExLGgsGF4jwj,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118
1,5DXq9lQxg6tF88XVzwOKGA,0.534,0.501,-8.764,0.0348,0.297000,0.000076,0.3420,0.909,177.437
2,1Y7u4vDBYUqGClwy917MUm,0.767,0.702,-5.963,0.0659,0.018100,0.023100,0.2320,0.704,119.985
3,0t0zDIeQykxGQUV5UHDgNA,0.679,0.913,-5.910,0.0675,0.000396,0.882000,0.0789,0.272,174.029
4,6riNnIsy4o3iIdIADdmcqO,0.650,0.825,-6.343,0.0714,0.000544,0.664000,0.1030,0.313,159.994
...,...,...,...,...,...,...,...,...,...,...
3251,6QE54DdZoeDixQSlxCzQwU,0.557,0.707,-3.977,0.0254,0.000279,0.001600,0.1450,0.557,105.067
3252,5qdHqzbMrlW1fmNfi9bi6a,0.747,0.719,-14.050,0.1130,0.303000,0.051200,0.2950,0.602,107.790
3253,2uV989rlzkgVr9kNzcBsi7,0.804,0.354,-10.875,0.3890,0.587000,0.000222,0.0714,0.780,170.091
3254,5mNV8Mz59bzyuQ53gTw0c0,0.596,0.661,-6.239,0.2260,0.310000,0.000000,0.0847,0.860,129.491


In [ ]:
#df_user_tracks has 3274 rows and 5 cols
df_user_tracks.shape

df_user_tracks

,Artist,Artist ID,Track,Track ID,Time Played At
0,Michael Franti & Spearhead,1mHuZMOP8FG5ip4yAb1vrB,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z
1,Dojo Cuts,0Vd8YQz8TYk2vSKEYVvIgL,I Can Give,5DXq9lQxg6tF88XVzwOKGA,2017-06-24T18:52:23.008Z
2,The Young Punx Cagedbaby D-Tale,3gkX03yW2WrvcDMp04TKjN 2sbZiktyjJtAAWs3LHeHzO ...,Loving You Baby,1Y7u4vDBYUqGClwy917MUm,2017-05-03T12:42:42.156Z
3,Mage,0q7PuJHIkWuHiHezAQFB6u,All The Time,0t0zDIeQykxGQUV5UHDgNA,2017-06-10T09:29:39.470Z
4,Bird of Prey Iguana,2mT2d7wMYhVOrQrjXMRWYS 66wBC351mvRza9Iw8pPn0M,Xenomorphic,6riNnIsy4o3iIdIADdmcqO,2017-06-27T00:24:42.441Z
...,...,...,...,...,...
3269,Nicole Willis & the Soul Investigators,1qR7R3OBeHqN7Dl12SBUMl,Holdin' On,5XekVKJWtC1d6C4VyQyLrQ,2017-04-06T23:16:23.576Z
3270,Nick Monaco,0fsDVdEDqPmn5CISll6OIQ,The Stalker,1zE1lRl7Xrt5fidhKzAktU,2017-04-07T13:30:40.276Z
3271,Mr President,0vxQClb5BOpAPfYZl1l0fR,Love and happiness,3c8UDT5mKtn81PCzinsA9Y,2017-04-24T08:42:06.233Z
3272,Bomba Estéreo,5n9bMYfz9qss2VOW89EVs2,Bosque,6jFI3qezWZdTaCZktJDhf9,2017-04-25T12:59:21.212Z


Comparing the df_user_tracks and the df_audio_features, we see that the former has 3274 rows and the latter has 3256 rows. This means that 18 tracks that appeared in the user's recently played list has not been accounted for. We investigated further to determine whether these tracks were ones that had been repeated or not.

We investigated the number of unique Track IDs that the user listened to.


In [ ]:
#Find the num of unique of Track IDs
len(df_user_tracks['Track ID'].unique())


2513

There are 2513 unique tracks that the user listens to. This means that the 18 tracks do not account for repeated tracks. We further investigated to see which tracks were left out when finding audio features.



In [ ]:
#Shape == (3274, 5)
df_user_tracks

#type(df_user_tracks['Track ID'][0])


,Artist,Artist ID,Track,Track ID,Time Played At
0,Michael Franti & Spearhead,1mHuZMOP8FG5ip4yAb1vrB,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z
1,Dojo Cuts,0Vd8YQz8TYk2vSKEYVvIgL,I Can Give,5DXq9lQxg6tF88XVzwOKGA,2017-06-24T18:52:23.008Z
2,The Young Punx Cagedbaby D-Tale,3gkX03yW2WrvcDMp04TKjN 2sbZiktyjJtAAWs3LHeHzO ...,Loving You Baby,1Y7u4vDBYUqGClwy917MUm,2017-05-03T12:42:42.156Z
3,Mage,0q7PuJHIkWuHiHezAQFB6u,All The Time,0t0zDIeQykxGQUV5UHDgNA,2017-06-10T09:29:39.470Z
4,Bird of Prey Iguana,2mT2d7wMYhVOrQrjXMRWYS 66wBC351mvRza9Iw8pPn0M,Xenomorphic,6riNnIsy4o3iIdIADdmcqO,2017-06-27T00:24:42.441Z
...,...,...,...,...,...
3269,Nicole Willis & the Soul Investigators,1qR7R3OBeHqN7Dl12SBUMl,Holdin' On,5XekVKJWtC1d6C4VyQyLrQ,2017-04-06T23:16:23.576Z
3270,Nick Monaco,0fsDVdEDqPmn5CISll6OIQ,The Stalker,1zE1lRl7Xrt5fidhKzAktU,2017-04-07T13:30:40.276Z
3271,Mr President,0vxQClb5BOpAPfYZl1l0fR,Love and happiness,3c8UDT5mKtn81PCzinsA9Y,2017-04-24T08:42:06.233Z
3272,Bomba Estéreo,5n9bMYfz9qss2VOW89EVs2,Bosque,6jFI3qezWZdTaCZktJDhf9,2017-04-25T12:59:21.212Z


In [ ]:
df_audio_features

#type(df_audio_features["Track ID"][0])

,Track ID,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,1SZThDcsBdExLGgsGF4jwj,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118
1,5DXq9lQxg6tF88XVzwOKGA,0.534,0.501,-8.764,0.0348,0.297000,0.000076,0.3420,0.909,177.437
2,1Y7u4vDBYUqGClwy917MUm,0.767,0.702,-5.963,0.0659,0.018100,0.023100,0.2320,0.704,119.985
3,0t0zDIeQykxGQUV5UHDgNA,0.679,0.913,-5.910,0.0675,0.000396,0.882000,0.0789,0.272,174.029
4,6riNnIsy4o3iIdIADdmcqO,0.650,0.825,-6.343,0.0714,0.000544,0.664000,0.1030,0.313,159.994
...,...,...,...,...,...,...,...,...,...,...
3251,6QE54DdZoeDixQSlxCzQwU,0.557,0.707,-3.977,0.0254,0.000279,0.001600,0.1450,0.557,105.067
3252,5qdHqzbMrlW1fmNfi9bi6a,0.747,0.719,-14.050,0.1130,0.303000,0.051200,0.2950,0.602,107.790
3253,2uV989rlzkgVr9kNzcBsi7,0.804,0.354,-10.875,0.3890,0.587000,0.000222,0.0714,0.780,170.091
3254,5mNV8Mz59bzyuQ53gTw0c0,0.596,0.661,-6.239,0.2260,0.310000,0.000000,0.0847,0.860,129.491


We conducted a left-join with df_user_tracks by track_id. This would preserve all the original tracks in the user tracks dataset and join the corresponding audio features.



In [ ]:
#left join  df_audio_features to df_user_tracks by track_id 
left_merged = df_user_tracks.merge(df_audio_features,
                        how="left", on=["Track ID"])

#Shape == (6485, 14)
left_merged



,Artist,Artist ID,Track,Track ID,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Michael Franti & Spearhead,1mHuZMOP8FG5ip4yAb1vrB,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322,0.009570,0.1820,0.893,171.118
1,Michael Franti & Spearhead,1mHuZMOP8FG5ip4yAb1vrB,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322,0.009570,0.1820,0.893,171.118
2,Michael Franti & Spearhead,1mHuZMOP8FG5ip4yAb1vrB,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322,0.009570,0.1820,0.893,171.118
3,Michael Franti & Spearhead,1mHuZMOP8FG5ip4yAb1vrB,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322,0.009570,0.1820,0.893,171.118
4,Dojo Cuts,0Vd8YQz8TYk2vSKEYVvIgL,I Can Give,5DXq9lQxg6tF88XVzwOKGA,2017-06-24T18:52:23.008Z,0.534,0.501,-8.764,0.0348,0.297,0.000076,0.3420,0.909,177.437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6480,Junior Kimbrough,03HEHGJoLPdARs4nrtUidr,Junior's Place,6iV8YSWlcwPPVxWryTMPFb,2017-06-19T08:48:28.087Z,0.776,0.757,-7.070,0.0616,0.685,0.652000,0.0804,0.704,114.136
6481,Junior Kimbrough,03HEHGJoLPdARs4nrtUidr,Junior's Place,6iV8YSWlcwPPVxWryTMPFb,2017-06-19T08:48:28.087Z,0.776,0.757,-7.070,0.0616,0.685,0.652000,0.0804,0.704,114.136
6482,Junior Kimbrough,03HEHGJoLPdARs4nrtUidr,Junior's Place,6iV8YSWlcwPPVxWryTMPFb,2017-06-19T08:48:28.087Z,0.776,0.757,-7.070,0.0616,0.685,0.652000,0.0804,0.704,114.136
6483,Junior Kimbrough,03HEHGJoLPdARs4nrtUidr,Junior's Place,6iV8YSWlcwPPVxWryTMPFb,2017-06-19T08:48:28.087Z,0.776,0.757,-7.070,0.0616,0.685,0.652000,0.0804,0.704,114.136


We did a left join to check that we were able to retrieve audio features for all the tracks in the dataset. This check is implementend afterwards by looking for NaN values for audio features for tracks from the original dataset. 

After left-joining the df_user_tracks dataframe and the df_audio_features dataframe, we found that duplicate rows exist.


In [ ]:
#Investigating duplicates in df_audio_features and df_user_tracks

#Original shape == (3256, 10)
df_audio_features.shape 

#Shape after dropping duplicate tracks == (2501, 10)
df_audio_features.drop_duplicates(subset=['Track ID']).shape


(2501, 10)

We know there are 2513 unique tracks in the user tracks dataset. This means that there is no audio features information present for 12 tracks from the original dataset. We further confirm this later and investigate which tracks have missing audio features.


Since a user may listen to a track more than once in a time-frame, for the dashboard dataframe, we remove duplicates based on "Time Played At" and not "Track ID", to ensure that the original patterns of playing the tracks are preserved for visualizations.


In [ ]:
#Creating dataframe for our dashboard
#Only keep the Artist, Track, Track ID, Audio Features and Time Played at for dashboard 
#Only keep rows with unique Time Played At 

df_dashboard = left_merged.drop(['Artist ID'], axis=1)
df_dashboard = df_dashboard.drop_duplicates(subset=['Time Played At'])

#Shape == (3274, 13)
df_dashboard 


,Artist,Track,Track ID,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Michael Franti & Spearhead,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118
4,Dojo Cuts,I Can Give,5DXq9lQxg6tF88XVzwOKGA,2017-06-24T18:52:23.008Z,0.534,0.501,-8.764,0.0348,0.297000,0.000076,0.3420,0.909,177.437
9,The Young Punx Cagedbaby D-Tale,Loving You Baby,1Y7u4vDBYUqGClwy917MUm,2017-05-03T12:42:42.156Z,0.767,0.702,-5.963,0.0659,0.018100,0.023100,0.2320,0.704,119.985
16,Mage,All The Time,0t0zDIeQykxGQUV5UHDgNA,2017-06-10T09:29:39.470Z,0.679,0.913,-5.910,0.0675,0.000396,0.882000,0.0789,0.272,174.029
17,Bird of Prey Iguana,Xenomorphic,6riNnIsy4o3iIdIADdmcqO,2017-06-27T00:24:42.441Z,0.650,0.825,-6.343,0.0714,0.000544,0.664000,0.1030,0.313,159.994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6473,Nicole Willis & the Soul Investigators,Holdin' On,5XekVKJWtC1d6C4VyQyLrQ,2017-04-06T23:16:23.576Z,0.435,0.841,-5.365,0.0356,0.079200,0.000078,0.1230,0.872,121.933
6474,Nick Monaco,The Stalker,1zE1lRl7Xrt5fidhKzAktU,2017-04-07T13:30:40.276Z,0.887,0.503,-8.190,0.0442,0.005850,0.503000,0.2200,0.882,121.014
6476,Mr President,Love and happiness,3c8UDT5mKtn81PCzinsA9Y,2017-04-24T08:42:06.233Z,0.878,0.804,-7.164,0.0361,0.038500,0.116000,0.0879,0.883,115.516
6477,Bomba Estéreo,Bosque,6jFI3qezWZdTaCZktJDhf9,2017-04-25T12:59:21.212Z,0.723,0.508,-12.418,0.0671,0.445000,0.830000,0.2060,0.715,120.971


In [ ]:
#Check for NaNs in df_dashboard
df_dashboard.isnull().sum().sum()


108

Since NaNs exist in the dataframe, we count how many there are and which columns they exist for.




In [ ]:
#Check which cols NaNs reside in 
df_dashboard.isnull().sum()

Artist               0
Track                0
Track ID             0
Time Played At       0
Danceability        12
Energy              12
Loudness            12
Speechiness         12
Acousticness        12
Instrumentalness    12
Liveness            12
Valence             12
Tempo               12
dtype: int64

We found that there exists a total of 108 NaNs in the df_dashboard dataframe. This corresponds to 12 NaNs for each of the 12 audio features. This may have resulted from us not being able to retrieve the data for 12 of the Track IDs from the Spotify Web API in the user_tracks dataframe. To investigate this, we try to see which Track IDs correspond to these NaNs and how many unique Track IDs exist out of that subset.

In [ ]:
#Select each row with atleast one NaN in left_merged df

rows_with_NaN = df_dashboard[df_dashboard.isnull().any(axis=1)]

#Number of Track IDs with missing audio features 
rows_with_NaN.shape[0]
rows_with_NaN

,Artist,Track,Track ID,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
398,Peanut Butter Wolf,Styles Crew Flows Beats,7iJFujOqpzWVRZBFrrioYW,2017-05-24T12:50:04.511Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
891,Diana Krall,Temptation,58mtR07okFcW7R2UUJHy0F,2017-05-12T15:48:17.297Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2766,Kölsch,Oma,65HKQWBLcZXibPENZvlQLv,2017-05-24T11:31:19.804Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3340,Tristan Aardvarkk,Daws of Perception - Original Mix,62GhghMtNOtUbFIWw4V0WY,2017-06-27T12:45:33.453Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3720,The Commercial Hippies,Live - Ital Remix,6wYSxfgyJWFyPFNiFQrX4O,2017-06-20T09:19:38.322Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4111,Idris Ackamoor The Pyramids,Rhapsody In Berlin,0rS1cEVBJ70VNBnMn1MkgH,2017-04-14T11:08:07.768Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4509,Sonic Species Altruism,Tokyo Sunrise - Altruism Remix,1ktWIGS7eMLzZqrVjSbPkW,2017-05-23T20:17:13.287Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4701,The Beach Boys,Good Vibrations - 2001 - Remaster,2oF7FZHIJbzjeEXZ3D0Ku4,2017-05-22T11:51:39.267Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4848,1200 Micrograms,Mescaline - Original Mix,3G3cpknRLszl9vH9knbGHt,2017-06-27T12:22:20.398Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5251,James Brown,The Boss,6Mg17BebEPPQiu628sGJGC,2017-04-24T13:05:56.564Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Our initial hypothesis turns out to be correct. We do not have audio features data for the 12 tracks above and this could be due to human error when retrieving the audio feautres from the API. To address this, we try once again to retrieve the audio features for the tracks above, using the Spotify Web API.


In [ ]:
#Create a str from the list and use it to retrieve the API curl command 
NaN_tracks_str = ''

for id in rows_with_NaN['Track ID']:
  NaN_tracks_str +=  id + ',' 

NaN_tracks_str


'7iJFujOqpzWVRZBFrrioYW,58mtR07okFcW7R2UUJHy0F,65HKQWBLcZXibPENZvlQLv,62GhghMtNOtUbFIWw4V0WY,6wYSxfgyJWFyPFNiFQrX4O,0rS1cEVBJ70VNBnMn1MkgH,1ktWIGS7eMLzZqrVjSbPkW,2oF7FZHIJbzjeEXZ3D0Ku4,3G3cpknRLszl9vH9knbGHt,6Mg17BebEPPQiu628sGJGC,6AgZ7vLUxmvB2D6fZe1quM,1XlluA8RIZkCKmmZcjppeg,'

In [ ]:
#Upload file of leftover audio tracks
uploaded_lft = files.upload()

Saving leftover_tracks.json to leftover_tracks (1).json


In [ ]:
file_name = "leftover_tracks.json"

#Decode JSON file
io.StringIO(uploaded_lft[file_name].decode("utf-8"))

#Parse JSON file into a Python dict
leftover_tracks_dict = json.loads(uploaded_lft[file_name].decode("utf-8"))

#Check if data uploaded correctly
leftover_tracks_dict

{'audio_features': [{'acousticness': 0.0875,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7iJFujOqpzWVRZBFrrioYW',
   'danceability': 0.513,
   'duration_ms': 351080,
   'energy': 0.61,
   'id': '7iJFujOqpzWVRZBFrrioYW',
   'instrumentalness': 0,
   'key': 1,
   'liveness': 0.224,
   'loudness': -6.67,
   'mode': 1,
   'speechiness': 0.331,
   'tempo': 90.582,
   'time_signature': 4,
   'track_href': 'https://api.spotify.com/v1/tracks/7iJFujOqpzWVRZBFrrioYW',
   'type': 'audio_features',
   'uri': 'spotify:track:7iJFujOqpzWVRZBFrrioYW',
   'valence': 0.694},
  {'acousticness': 0.287,
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/58mtR07okFcW7R2UUJHy0F',
   'danceability': 0.753,
   'duration_ms': 267720,
   'energy': 0.189,
   'id': '58mtR07okFcW7R2UUJHy0F',
   'instrumentalness': 0.00301,
   'key': 5,
   'liveness': 0.0854,
   'loudness': -14.235,
   'mode': 0,
   'speechiness': 0.0352,
   'tempo': 104.949,
   'time_signature': 4,
   'track_href': 'htt

In [ ]:
#Create a dataframe out of the leftover_tracks_dict 

#init
track_id_lft = []
danceability = []
energy = []
loudness = [] 
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = [] 

#Iterate over the dictionary and create separate arrays for each feature 
for i in range(0, rows_with_NaN.shape[0]):
  #Add elements to the lists 
  danceability.append(leftover_tracks_dict['audio_features'][i]['danceability'])
  energy.append(leftover_tracks_dict['audio_features'][i]['energy'])
  loudness.append(leftover_tracks_dict['audio_features'][i]['loudness'])
  speechiness.append(leftover_tracks_dict['audio_features'][i]['speechiness'])
  acousticness.append(leftover_tracks_dict['audio_features'][i]['acousticness'])
  instrumentalness.append(leftover_tracks_dict['audio_features'][i]['instrumentalness'])
  liveness.append(leftover_tracks_dict['audio_features'][i]['liveness'])
  valence.append(leftover_tracks_dict['audio_features'][i]['valence'])
  tempo.append(leftover_tracks_dict['audio_features'][i]['tempo'])
  track_id_lft.append(leftover_tracks_dict['audio_features'][i]['id'])

#create dataframe using lists
df_leftover_tracks = pd.DataFrame(list(zip(track_id_lft, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo)),
               columns =['Track ID','Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo'])

In [ ]:
df_leftover_tracks

,Track ID,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,7iJFujOqpzWVRZBFrrioYW,0.513,0.610,-6.670,0.3310,0.087500,0.000000,0.2240,0.6940,90.582
1,58mtR07okFcW7R2UUJHy0F,0.753,0.189,-14.235,0.0352,0.287000,0.003010,0.0854,0.5790,104.949
2,65HKQWBLcZXibPENZvlQLv,0.757,0.698,-10.244,0.0465,0.000321,0.516000,0.2250,0.4990,125.015
3,62GhghMtNOtUbFIWw4V0WY,0.599,0.947,-5.439,0.0431,0.000014,0.699000,0.3460,0.2980,145.998
4,6wYSxfgyJWFyPFNiFQrX4O,0.681,0.911,-7.270,0.0422,0.000021,0.864000,0.0807,0.0968,144.020
5,0rS1cEVBJ70VNBnMn1MkgH,0.577,0.861,-4.373,0.1750,0.157000,0.473000,0.1490,0.6920,90.847
6,1ktWIGS7eMLzZqrVjSbPkW,0.651,0.882,-6.129,0.0566,0.000061,0.801000,0.1680,0.5300,142.983
7,2oF7FZHIJbzjeEXZ3D0Ku4,0.403,0.495,-6.888,0.0312,0.387000,0.000008,0.1470,0.3800,132.912
8,3G3cpknRLszl9vH9knbGHt,0.655,0.959,-7.683,0.0960,0.050100,0.878000,0.2660,0.3430,148.009
9,6Mg17BebEPPQiu628sGJGC,0.746,0.484,-9.614,0.0434,0.602000,0.008240,0.0576,0.6060,97.923


Now we update df_dashboard to add the leftover tracks and their information.


In [ ]:
#Check for NaNs in df_dashboard --- 108 NaNs still exist

df_dashboard.isnull().sum()

Artist               0
Track                0
Track ID             0
Time Played At       0
Danceability        12
Energy              12
Loudness            12
Speechiness         12
Acousticness        12
Instrumentalness    12
Liveness            12
Valence             12
Tempo               12
dtype: int64

In [ ]:
#Update dashboard NaNs with audio features
dashboard_edit = df_dashboard.set_index('Track ID')
leftover_tracks_edit = df_leftover_tracks.set_index('Track ID')
dashboard_edit.update(leftover_tracks_edit)


In [ ]:
dashboard_edit.reset_index(inplace = True)

In [ ]:
dashboard_edit

,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,1SZThDcsBdExLGgsGF4jwj,Michael Franti & Spearhead,People In Tha Middle,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118
1,5DXq9lQxg6tF88XVzwOKGA,Dojo Cuts,I Can Give,2017-06-24T18:52:23.008Z,0.534,0.501,-8.764,0.0348,0.297000,0.000076,0.3420,0.909,177.437
2,1Y7u4vDBYUqGClwy917MUm,The Young Punx Cagedbaby D-Tale,Loving You Baby,2017-05-03T12:42:42.156Z,0.767,0.702,-5.963,0.0659,0.018100,0.023100,0.2320,0.704,119.985
3,0t0zDIeQykxGQUV5UHDgNA,Mage,All The Time,2017-06-10T09:29:39.470Z,0.679,0.913,-5.910,0.0675,0.000396,0.882000,0.0789,0.272,174.029
4,6riNnIsy4o3iIdIADdmcqO,Bird of Prey Iguana,Xenomorphic,2017-06-27T00:24:42.441Z,0.650,0.825,-6.343,0.0714,0.000544,0.664000,0.1030,0.313,159.994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269,5XekVKJWtC1d6C4VyQyLrQ,Nicole Willis & the Soul Investigators,Holdin' On,2017-04-06T23:16:23.576Z,0.435,0.841,-5.365,0.0356,0.079200,0.000078,0.1230,0.872,121.933
3270,1zE1lRl7Xrt5fidhKzAktU,Nick Monaco,The Stalker,2017-04-07T13:30:40.276Z,0.887,0.503,-8.190,0.0442,0.005850,0.503000,0.2200,0.882,121.014
3271,3c8UDT5mKtn81PCzinsA9Y,Mr President,Love and happiness,2017-04-24T08:42:06.233Z,0.878,0.804,-7.164,0.0361,0.038500,0.116000,0.0879,0.883,115.516
3272,6jFI3qezWZdTaCZktJDhf9,Bomba Estéreo,Bosque,2017-04-25T12:59:21.212Z,0.723,0.508,-12.418,0.0671,0.445000,0.830000,0.2060,0.715,120.971


In [ ]:
leftover_tracks_edit.reset_index(inplace= True)

In [ ]:
leftover_tracks_edit

,Track ID,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,7iJFujOqpzWVRZBFrrioYW,0.513,0.610,-6.670,0.3310,0.087500,0.000000,0.2240,0.6940,90.582
1,58mtR07okFcW7R2UUJHy0F,0.753,0.189,-14.235,0.0352,0.287000,0.003010,0.0854,0.5790,104.949
2,65HKQWBLcZXibPENZvlQLv,0.757,0.698,-10.244,0.0465,0.000321,0.516000,0.2250,0.4990,125.015
3,62GhghMtNOtUbFIWw4V0WY,0.599,0.947,-5.439,0.0431,0.000014,0.699000,0.3460,0.2980,145.998
4,6wYSxfgyJWFyPFNiFQrX4O,0.681,0.911,-7.270,0.0422,0.000021,0.864000,0.0807,0.0968,144.020
5,0rS1cEVBJ70VNBnMn1MkgH,0.577,0.861,-4.373,0.1750,0.157000,0.473000,0.1490,0.6920,90.847
6,1ktWIGS7eMLzZqrVjSbPkW,0.651,0.882,-6.129,0.0566,0.000061,0.801000,0.1680,0.5300,142.983
7,2oF7FZHIJbzjeEXZ3D0Ku4,0.403,0.495,-6.888,0.0312,0.387000,0.000008,0.1470,0.3800,132.912
8,3G3cpknRLszl9vH9knbGHt,0.655,0.959,-7.683,0.0960,0.050100,0.878000,0.2660,0.3430,148.009
9,6Mg17BebEPPQiu628sGJGC,0.746,0.484,-9.614,0.0434,0.602000,0.008240,0.0576,0.6060,97.923


In [ ]:
#Ensure shape was preserved
dashboard_edit.shape

(3274, 13)

In [ ]:
#Check NaNs were removed
dashboard_edit.isnull().sum()

Track ID            0
Artist              0
Track               0
Time Played At      0
Danceability        0
Energy              0
Loudness            0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [ ]:
df_dashboard = dashboard_edit

In [ ]:
df_dashboard

,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,1SZThDcsBdExLGgsGF4jwj,Michael Franti & Spearhead,People In Tha Middle,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118
1,5DXq9lQxg6tF88XVzwOKGA,Dojo Cuts,I Can Give,2017-06-24T18:52:23.008Z,0.534,0.501,-8.764,0.0348,0.297000,0.000076,0.3420,0.909,177.437
2,1Y7u4vDBYUqGClwy917MUm,The Young Punx Cagedbaby D-Tale,Loving You Baby,2017-05-03T12:42:42.156Z,0.767,0.702,-5.963,0.0659,0.018100,0.023100,0.2320,0.704,119.985
3,0t0zDIeQykxGQUV5UHDgNA,Mage,All The Time,2017-06-10T09:29:39.470Z,0.679,0.913,-5.910,0.0675,0.000396,0.882000,0.0789,0.272,174.029
4,6riNnIsy4o3iIdIADdmcqO,Bird of Prey Iguana,Xenomorphic,2017-06-27T00:24:42.441Z,0.650,0.825,-6.343,0.0714,0.000544,0.664000,0.1030,0.313,159.994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269,5XekVKJWtC1d6C4VyQyLrQ,Nicole Willis & the Soul Investigators,Holdin' On,2017-04-06T23:16:23.576Z,0.435,0.841,-5.365,0.0356,0.079200,0.000078,0.1230,0.872,121.933
3270,1zE1lRl7Xrt5fidhKzAktU,Nick Monaco,The Stalker,2017-04-07T13:30:40.276Z,0.887,0.503,-8.190,0.0442,0.005850,0.503000,0.2200,0.882,121.014
3271,3c8UDT5mKtn81PCzinsA9Y,Mr President,Love and happiness,2017-04-24T08:42:06.233Z,0.878,0.804,-7.164,0.0361,0.038500,0.116000,0.0879,0.883,115.516
3272,6jFI3qezWZdTaCZktJDhf9,Bomba Estéreo,Bosque,2017-04-25T12:59:21.212Z,0.723,0.508,-12.418,0.0671,0.445000,0.830000,0.2060,0.715,120.971


We create a txt file that will act as an input to our Python script that uses the Genius API and BeautifulSoup to scrape the song lyrics. We use artistName and songName for each song to scrape the lyrics, and we keep the Track ID as an identifier, to merge the dataframe with emotions with our current dataframe


In [ ]:
import re

#fn that gets the artistName, songName, and trackID for each track in df, and writes it to a txt file
# @param: df = pandas dataframe, filename = str 

def get_artist_and_song(df, filename):
  
  with open(filename,'w',encoding = 'utf-8') as f:
    for i in range(df.shape[0]):
      #set artist name
      artistName = df['Artist'][i]
    
      #set song name
      songName = df['Track'][i]
      #set trackID
      trackID = df['Track ID'][i]

      #remove unnecessary info after hyphens (e.g. - 1998 remastered remix)
      songName = re.sub(r'-...*',"",songName)
      artistName = re.sub(r'-...*',"",artistName)

      #if any feat. artists, remove them from artistName
      if re.search(r"feat...*[\)\]]", songName):
  
        x = re.search(r"feat...*[\)\]]", songName)
        y = x[0]
        y = re.sub("feat. ", "", x[0])
        y = re.sub('[\(\[\)\]]', "", y)
        artistName = re.sub(y, "", artistName)
 

      #remove any identifiers like (feat...), (Part 1 & 2), "-remastered remix" etc'
      songName = re.sub('[\(\[].*?[\)\]]', '', songName)
      artistName = re.sub('[\(\[].*?[\)\]]', '', artistName)
   

      #replace language specific chars with English chars 
      artistName = unidecode(artistName)
      songName = unidecode(songName)
    
      #remove punctuation from names 
      songName = re.sub(r'[.\,\"\'\-\?\:\!\;]',"",songName)
      artistName = re.sub(r'[.\,\"\'\-\?\:\!\;]',"",artistName)
   

      #remove leading and trailing space chars 
      songName = songName.strip()
      artistName = artistName.strip()


    #problems - clean bandit zara larsson for "artist name", rihanna 3LAU for artist name 
      #write to file 
      f.write(artistName + "," + songName + "," + trackID + "\n")
    f.close()


In [ ]:
#fn that resets the index of any dataframe
# @params: df = dataframe

def reindex_df(df):
  #Re-index june_data 
  df.reset_index(inplace=True)

  #drop index col 
  if 'index' in df.columns:
    df.drop(labels = ['index'], axis=1, inplace = True)
  if '0' in df.columns:
    df.drop(labels=['0'], axis =1, inplace = True)
  return df


In [ ]:

filename = "artistAndSongName.txt"

#Ensure df is indexed to be 0,1,...
df_dashboard = reindex_df(df_dashboard)

get_artist_and_song(df_dashboard, filename)


In [ ]:
with open('artistAndSongName.txt', 'r') as testwritefile:
    print(testwritefile.read())

Michael Franti & Spearhead,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj
Dojo Cuts,I Can Give,5DXq9lQxg6tF88XVzwOKGA
The Young Punx Cagedbaby D,Loving You Baby,1Y7u4vDBYUqGClwy917MUm
Mage,All The Time,0t0zDIeQykxGQUV5UHDgNA
Bird of Prey Iguana,Xenomorphic,6riNnIsy4o3iIdIADdmcqO
Altruism,Gratitude,0yXij8RLCsp2Ir3Wkv9bCS
Quincy Jones,Let The Good Times Roll,5s9wI2Jijxp5sDyL9ltKmR
Madlib,Slims Return,2P5mABk27DkOT8Gi2NbTpO
William DeVaughn,Be Thankful for What You Got,7pN8GMLw3prWFn5nvJRM6O
Kolsch Troels Abrahamsen,All that matters,5hGBJKU8aJaZPVSV7NxoTp
Ginuwine,Pony,6mz1fBdKATx6qP4oP1I65G
Entheogenic,Ground Luminosity,5HOyaBTI0ypXjyowSRKeHj
Zen Mechanics,Modified,1GIeXkjmmoYVFoQpbV6sXD
Count Basie,A Hard Days Night,2PMZRVTJlDj2DokOxYnmnY
Junior Kimbrough,Sad Days Lonely Nights,1NaRFCn2s3Rb75ghDIfBuR
Infected Mushroom,Devil,61HCD3QERj7kxp4qHSzGiF
Pernett,Hoy Es Manana,2Ds6mEvudpm8tlBY8s6pPl
Dinah Washington,Mad About The Boy,15VvVzNAne1ndguY5Rh8ji
Public Enemy,Give It Up,2uWoy80sBB8a59nh0c

In [ ]:
files.download('artistAndSongName.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Sentiment Analysis of Lyrics**

We used the artistAndSongName.txt file as input to the Python script, lyric_EmotionCollect.py. This script takes in the artist name and song title for each song in the above dataset and web scrapes the corresponding lyrics using the Genius API and BeautifulSoup library in Python.

Then, we tokenized the lyrics, excluded stop words from tokens, excluded one-letter and two-letter words (as we believed they are also mostly stop words) and finally conducted lemmatization on the tokens. This is because we believed the NRC lexicon dictionary is more likely to have the lemmatized versions of words.

We then conducted NRC lexicon analysis on every remaining token and retrieved the non-zero emotion scores for each song. 

The results are as follows:


In [ ]:
#Retrieve csv file containing dataset of artist name, song name, trackID and NRC emotion scores for all songs 

#root_path = 'gdrive/MyDrive/Menon_Labs/code_and_data'
df_emotions = pd.read_csv("gdrive/MyDrive/Menon_Labs/code_and_data/emotions_from_lyrics.csv")

df_emotions




,Artist,Track,Track ID,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,Michael Franti and Spearhead,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj\n,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799
1,Dojo Cuts,I Can Give,5DXq9lQxg6tF88XVzwOKGA\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Young Punx Cagedbaby D,Loving You Baby,1Y7u4vDBYUqGClwy917MUm\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mage,All The Time,0t0zDIeQykxGQUV5UHDgNA\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bird of Prey Iguana,Xenomorphic,6riNnIsy4o3iIdIADdmcqO\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269,Nicole Willis and the Soul Investigators,Holdin On,5XekVKJWtC1d6C4VyQyLrQ\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270,Nick Monaco,The Stalker,1zE1lRl7Xrt5fidhKzAktU\n,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3271,Mr President,Love and happiness,3c8UDT5mKtn81PCzinsA9Y\n,0.011700,0.130556,0.000000,0.024647,0.000000,0.000000,0.011700,0.000000,0.133333,0.085833
3272,Bomba Estereo,Bosque,6jFI3qezWZdTaCZktJDhf9\n,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
uploaded = files.upload()

Saving emotions_from_lyrics.csv to emotions_from_lyrics.csv


In [ ]:
df_emotions = pd.read_csv(io.BytesIO(uploaded['emotions_from_lyrics.csv']))
df_emotions

,Artist,Track,Track ID,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,Michael Franti and Spearhead,People In Tha Middle,1SZThDcsBdExLGgsGF4jwj\n,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799
1,Dojo Cuts,I Can Give,5DXq9lQxg6tF88XVzwOKGA\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Young Punx Cagedbaby D,Loving You Baby,1Y7u4vDBYUqGClwy917MUm\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mage,All The Time,0t0zDIeQykxGQUV5UHDgNA\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bird of Prey Iguana,Xenomorphic,6riNnIsy4o3iIdIADdmcqO\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269,Nicole Willis and the Soul Investigators,Holdin On,5XekVKJWtC1d6C4VyQyLrQ\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270,Nick Monaco,The Stalker,1zE1lRl7Xrt5fidhKzAktU\n,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3271,Mr President,Love and happiness,3c8UDT5mKtn81PCzinsA9Y\n,0.011700,0.130556,0.000000,0.024647,0.000000,0.000000,0.011700,0.000000,0.133333,0.085833
3272,Bomba Estereo,Bosque,6jFI3qezWZdTaCZktJDhf9\n,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


When creating the txt file, we added endline characters to ensure the track information for each track remained on separate lines. However, since the endline char remains appended to the Track ID, we remove it.


In [ ]:
#Removing "\n" from Track ID for all Track IDs
for i in range(0, df_emotions.shape[0]):
  df_emotions.loc[i, "Track ID"] = re.sub("\n", "", df_emotions.loc[i, "Track ID"])


We remove the Artist and the Track columns from the df_emotions as those identifiers have been transformed from their original versions to scrape the lyrics.

In [ ]:
#Drop "Artist" and "Track" cols from df_emotions
df_emotions.drop(labels = ["Artist", "Track"], axis=1, inplace=True)

In [ ]:

df_emotions.isnull().sum()

Track ID           0
Positive        2010
Negative        2010
Anger           2010
Anticipation    2010
Disgust         2010
Fear            2010
Joy             2010
Sadness         2010
Surprise        2010
Trust           2010
dtype: int64

We drop the songs that did not return any values from the Genius API. We then remove duplicates so there is a one-to-one match when updating df_dashboard with the emotions.

In [ ]:
#Drop songs that did not return any lyrics from Genius API
# A total of 1264 tracks returned lyrics using the Genius API 
df_emotions.dropna(inplace=True)
df_emotions

,Track ID,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,1SZThDcsBdExLGgsGF4jwj,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799
6,5s9wI2Jijxp5sDyL9ltKmR,0.028367,0.218333,0.070833,0.009764,0.144444,0.077778,0.013127,0.091667,0.014261,0.039677
7,2P5mABk27DkOT8Gi2NbTpO,0.375000,0.200000,0.200000,0.250000,0.200000,0.200000,0.250000,0.000000,0.200000,0.375000
10,6mz1fBdKATx6qP4oP1I65G,0.035914,0.112500,0.266667,0.000000,0.200000,0.266667,0.045882,0.205556,1.000000,0.555556
13,2PMZRVTJlDj2DokOxYnmnY,0.150747,0.100000,0.077778,0.077778,0.100000,0.091667,0.077778,0.091667,0.077778,0.077778
...,...,...,...,...,...,...,...,...,...,...,...
3259,3mSQSO6mtVshdnJD3koXTM,0.186111,0.027362,0.350000,0.172946,0.350000,0.057646,0.186111,0.039783,0.666667,0.186111
3261,4KsikATrwteXcV7FrxMPg4,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000
3270,1zE1lRl7Xrt5fidhKzAktU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3271,3c8UDT5mKtn81PCzinsA9Y,0.011700,0.130556,0.000000,0.024647,0.000000,0.000000,0.011700,0.000000,0.133333,0.085833


In [ ]:
#Remove duplicates 
#There exist 942 unique tracks that returned lyrics and had emotion scores
df_emotions.drop_duplicates(subset=['Track ID'], inplace=True)
df_emotions



,Track ID,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,1SZThDcsBdExLGgsGF4jwj,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799
6,5s9wI2Jijxp5sDyL9ltKmR,0.028367,0.218333,0.070833,0.009764,0.144444,0.077778,0.013127,0.091667,0.014261,0.039677
7,2P5mABk27DkOT8Gi2NbTpO,0.375000,0.200000,0.200000,0.250000,0.200000,0.200000,0.250000,0.000000,0.200000,0.375000
10,6mz1fBdKATx6qP4oP1I65G,0.035914,0.112500,0.266667,0.000000,0.200000,0.266667,0.045882,0.205556,1.000000,0.555556
13,2PMZRVTJlDj2DokOxYnmnY,0.150747,0.100000,0.077778,0.077778,0.100000,0.091667,0.077778,0.091667,0.077778,0.077778
...,...,...,...,...,...,...,...,...,...,...,...
3257,7km1tntASwmtyzGr8vaT0e,0.022309,0.215278,0.333333,0.416667,0.305556,1.000000,0.048669,0.000000,0.250000,0.033909
3259,3mSQSO6mtVshdnJD3koXTM,0.186111,0.027362,0.350000,0.172946,0.350000,0.057646,0.186111,0.039783,0.666667,0.186111
3261,4KsikATrwteXcV7FrxMPg4,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000
3271,3c8UDT5mKtn81PCzinsA9Y,0.011700,0.130556,0.000000,0.024647,0.000000,0.000000,0.011700,0.000000,0.133333,0.085833


We merge df_emotions to df_dashboard to get a dataframe that shows all audio features and emotions from lyrics for each song. We then re-drop the songs with NaN values and create df_model with unique songs and their features. This will be used for our machine learning model.


In [ ]:
#Add emotions to df_dashboard 
#df_dashboard = df_dashboard.set_index('Track ID')
#df_emotions = df_emotions.set_index('Track ID')

df_dashboard = df_dashboard.merge(df_emotions,
                        how="left", on=["Track ID"])



In [ ]:
df_dashboard

,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,1SZThDcsBdExLGgsGF4jwj,Michael Franti & Spearhead,People In Tha Middle,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799
1,5DXq9lQxg6tF88XVzwOKGA,Dojo Cuts,I Can Give,2017-06-24T18:52:23.008Z,0.534,0.501,-8.764,0.0348,0.297000,0.000076,0.3420,0.909,177.437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1Y7u4vDBYUqGClwy917MUm,The Young Punx Cagedbaby D-Tale,Loving You Baby,2017-05-03T12:42:42.156Z,0.767,0.702,-5.963,0.0659,0.018100,0.023100,0.2320,0.704,119.985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0t0zDIeQykxGQUV5UHDgNA,Mage,All The Time,2017-06-10T09:29:39.470Z,0.679,0.913,-5.910,0.0675,0.000396,0.882000,0.0789,0.272,174.029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6riNnIsy4o3iIdIADdmcqO,Bird of Prey Iguana,Xenomorphic,2017-06-27T00:24:42.441Z,0.650,0.825,-6.343,0.0714,0.000544,0.664000,0.1030,0.313,159.994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269,5XekVKJWtC1d6C4VyQyLrQ,Nicole Willis & the Soul Investigators,Holdin' On,2017-04-06T23:16:23.576Z,0.435,0.841,-5.365,0.0356,0.079200,0.000078,0.1230,0.872,121.933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270,1zE1lRl7Xrt5fidhKzAktU,Nick Monaco,The Stalker,2017-04-07T13:30:40.276Z,0.887,0.503,-8.190,0.0442,0.005850,0.503000,0.2200,0.882,121.014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3271,3c8UDT5mKtn81PCzinsA9Y,Mr President,Love and happiness,2017-04-24T08:42:06.233Z,0.878,0.804,-7.164,0.0361,0.038500,0.116000,0.0879,0.883,115.516,0.011700,0.130556,0.000000,0.024647,0.000000,0.000000,0.011700,0.000000,0.133333,0.085833
3272,6jFI3qezWZdTaCZktJDhf9,Bomba Estéreo,Bosque,2017-04-25T12:59:21.212Z,0.723,0.508,-12.418,0.0671,0.445000,0.830000,0.2060,0.715,120.971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
#Drop songs that do not have emotion values 
df_dashboard.dropna(inplace=True)
df_dashboard

,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,1SZThDcsBdExLGgsGF4jwj,Michael Franti & Spearhead,People In Tha Middle,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799
6,5s9wI2Jijxp5sDyL9ltKmR,Quincy Jones,Let The Good Times Roll,2017-04-04T12:26:10.792Z,0.584,0.715,-6.505,0.1180,0.318000,0.000001,0.3940,0.754,106.556,0.028367,0.218333,0.070833,0.009764,0.144444,0.077778,0.013127,0.091667,0.014261,0.039677
7,2P5mABk27DkOT8Gi2NbTpO,Madlib,Slim's Return,2017-06-24T18:22:51.015Z,0.508,0.668,-7.281,0.2210,0.041200,0.032700,0.3040,0.548,91.590,0.375000,0.200000,0.200000,0.250000,0.200000,0.200000,0.250000,0.000000,0.200000,0.375000
8,7pN8GMLw3prWFn5nvJRM6O,William DeVaughn,Be Thankful for What You Got (Pt. 1 & Pt. 2),2017-04-18T09:40:40.808Z,0.791,0.420,-13.289,0.0832,0.413000,0.116000,0.1110,0.840,93.225,0.033723,0.000000,0.000000,0.071296,0.000000,0.000000,0.500000,0.000000,0.000000,0.042482
10,6mz1fBdKATx6qP4oP1I65G,Ginuwine,Pony,2017-06-09T08:03:57.958Z,0.749,0.605,-9.359,0.0860,0.001860,0.038100,0.1150,0.966,142.024,0.035914,0.112500,0.266667,0.000000,0.200000,0.266667,0.045882,0.205556,1.000000,0.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,3mSQSO6mtVshdnJD3koXTM,Alice Russell,Hurry On Now,2017-05-09T18:14:31.885Z,0.618,0.442,-7.016,0.0306,0.662000,0.000040,0.3250,0.268,83.390,0.186111,0.027362,0.350000,0.172946,0.350000,0.057646,0.186111,0.039783,0.666667,0.186111
3261,4KsikATrwteXcV7FrxMPg4,Panda Dub,J'finirai Seul Dans Un Coquillage,2017-04-21T19:49:37.185Z,0.505,0.660,-4.804,0.0349,0.000651,0.342000,0.0971,0.254,177.951,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000
3270,1zE1lRl7Xrt5fidhKzAktU,Nick Monaco,The Stalker,2017-04-07T13:30:40.276Z,0.887,0.503,-8.190,0.0442,0.005850,0.503000,0.2200,0.882,121.014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3271,3c8UDT5mKtn81PCzinsA9Y,Mr President,Love and happiness,2017-04-24T08:42:06.233Z,0.878,0.804,-7.164,0.0361,0.038500,0.116000,0.0879,0.883,115.516,0.011700,0.130556,0.000000,0.024647,0.000000,0.000000,0.011700,0.000000,0.133333,0.085833


In [ ]:
#added time of day, day of week, and month columns
import datetime
df_dashboard['Time'] = df_dashboard['Time Played At'].str[11:16]
df_dashboard['Date'] = df_dashboard['Time Played At'].str[0:10]
df_dashboard['Date'] = pd.to_datetime(df_dashboard['Date'])
df_dashboard['Day Of Week'] = df_dashboard['Date'].dt.day_name()
df_dashboard['Month'] = df_dashboard['Date'].dt.month_name()
df_dashboard['Day of Month'] = df_dashboard['Date'].dt.day
df_dashboard['Time'] = pd.to_datetime(df_dashboard['Time'])

#split into 6 hour subsets
def get_tod(r):
  now = datetime.datetime.now()
  morning_start = now.replace(hour=5, minute=59, second=0, microsecond=0)
  morning_end = now.replace(hour=12, minute=0, second=0, microsecond=0)
  afternoon_start = now.replace(hour=11, minute=59, second=0, microsecond=0)
  afternoon_end = now.replace(hour=18, minute=0, second=0, microsecond=0)
  evening_start = now.replace(hour=17, minute=59, second=0, microsecond=0)
  evening_end = now.replace(hour=23, minute=59, second=59, microsecond=59)

  if r['Time'] > morning_start and r['Time'] < morning_end:
    return 'Morning'
  elif r['Time'] > afternoon_start and r['Time'] < afternoon_end:
    return 'Afternoon'
  elif r['Time'] >  evening_start and r['Time'] < evening_end:
    return 'Evening'

df_dashboard['Time Of Day'] = df_dashboard.apply(get_tod, axis=1)

In [ ]:
df_dashboard = df_dashboard.drop(columns=['Date','Time'])

In [ ]:
df_dashboard

,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Day Of Week,Month,Day of Month,Time Of Day
0,1SZThDcsBdExLGgsGF4jwj,Michael Franti & Spearhead,People In Tha Middle,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.322000,0.009570,0.1820,0.893,171.118,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799,Saturday,June,24,Evening
6,5s9wI2Jijxp5sDyL9ltKmR,Quincy Jones,Let The Good Times Roll,2017-04-04T12:26:10.792Z,0.584,0.715,-6.505,0.1180,0.318000,0.000001,0.3940,0.754,106.556,0.028367,0.218333,0.070833,0.009764,0.144444,0.077778,0.013127,0.091667,0.014261,0.039677,Tuesday,April,4,Afternoon
7,2P5mABk27DkOT8Gi2NbTpO,Madlib,Slim's Return,2017-06-24T18:22:51.015Z,0.508,0.668,-7.281,0.2210,0.041200,0.032700,0.3040,0.548,91.590,0.375000,0.200000,0.200000,0.250000,0.200000,0.200000,0.250000,0.000000,0.200000,0.375000,Saturday,June,24,Evening
8,7pN8GMLw3prWFn5nvJRM6O,William DeVaughn,Be Thankful for What You Got (Pt. 1 & Pt. 2),2017-04-18T09:40:40.808Z,0.791,0.420,-13.289,0.0832,0.413000,0.116000,0.1110,0.840,93.225,0.033723,0.000000,0.000000,0.071296,0.000000,0.000000,0.500000,0.000000,0.000000,0.042482,Tuesday,April,18,Morning
10,6mz1fBdKATx6qP4oP1I65G,Ginuwine,Pony,2017-06-09T08:03:57.958Z,0.749,0.605,-9.359,0.0860,0.001860,0.038100,0.1150,0.966,142.024,0.035914,0.112500,0.266667,0.000000,0.200000,0.266667,0.045882,0.205556,1.000000,0.555556,Friday,June,9,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,3mSQSO6mtVshdnJD3koXTM,Alice Russell,Hurry On Now,2017-05-09T18:14:31.885Z,0.618,0.442,-7.016,0.0306,0.662000,0.000040,0.3250,0.268,83.390,0.186111,0.027362,0.350000,0.172946,0.350000,0.057646,0.186111,0.039783,0.666667,0.186111,Tuesday,May,9,Evening
3261,4KsikATrwteXcV7FrxMPg4,Panda Dub,J'finirai Seul Dans Un Coquillage,2017-04-21T19:49:37.185Z,0.505,0.660,-4.804,0.0349,0.000651,0.342000,0.0971,0.254,177.951,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000,Friday,April,21,Evening
3270,1zE1lRl7Xrt5fidhKzAktU,Nick Monaco,The Stalker,2017-04-07T13:30:40.276Z,0.887,0.503,-8.190,0.0442,0.005850,0.503000,0.2200,0.882,121.014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Friday,April,7,Afternoon
3271,3c8UDT5mKtn81PCzinsA9Y,Mr President,Love and happiness,2017-04-24T08:42:06.233Z,0.878,0.804,-7.164,0.0361,0.038500,0.116000,0.0879,0.883,115.516,0.011700,0.130556,0.000000,0.024647,0.000000,0.000000,0.011700,0.000000,0.133333,0.085833,Monday,April,24,Morning


In [ ]:
#download df_dashboard to prepare lyricCollect values
df_dashboard.to_csv('df.csv')
files.download('df.csv')

# Finding Top Emotion of Month:

In [ ]:
#Create subsets of each month
april_data = df_dashboard[df_dashboard["Month"] == "April"]
may_data = df_dashboard[df_dashboard["Month"] == "May"]
june_data = df_dashboard[df_dashboard["Month"] == "June"]
all_months_data = [april_data, may_data, june_data]
emotions = ["Anger","Anticipation",	"Disgust",	"Fear",	"Joy", "Sadness",	"Surprise",	"Trust"]
month_title= ["April", "May", "June"]


#finding the overall top emotion for each month
emotion_value = []
overall_emotion = ""
loc = 0
for month in all_months_data:
  for emotion in emotions:
    emotion_value.append(month[emotion].sum())
  emotion_loc = emotion_value.index(max(emotion_value))
  print(month_title[loc]+" top emotion:", emotions[emotion_loc])
  loc+=1
  emotion_value = []


April top emotion: Anticipation
May top emotion: Anticipation
June top emotion: Surprise


#Examining the data for the month of June

We examined the month of June and used its data as a focus of our dashboard. 

First, we examine how many data points exist for the month of June. Then we extract a separate dataframe for each day.

In [ ]:
# There are 220 tracks that the user listened to in June
june_data.shape

(220, 27)

In [ ]:
#Extracting values for 24th June
june_data.loc[june_data['Day of Month'] == 24]


,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Day Of Week,Month,Day of Month,Time Of Day
0,1SZThDcsBdExLGgsGF4jwj,Michael Franti & Spearhead,People In Tha Middle,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.32200,0.009570,0.1820,0.893,171.118,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799,Saturday,June,24,Evening
7,2P5mABk27DkOT8Gi2NbTpO,Madlib,Slim's Return,2017-06-24T18:22:51.015Z,0.508,0.668,-7.281,0.2210,0.04120,0.032700,0.3040,0.548,91.590,0.375000,0.200000,0.200000,0.250000,0.200000,0.200000,0.250000,0.000000,0.200000,0.375000,Saturday,June,24,Evening
270,7f6335NYzOYXaEGQsQQX69,Lowell Fulson,Why Don't We Do It In The Road,2017-06-24T23:09:19.180Z,0.508,0.503,-14.461,0.0300,0.39000,0.001930,0.0840,0.904,176.344,0.035914,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035914,Saturday,June,24,Evening
559,5uoM3DBcDiLvNaf5whIx1v,Electric Guest,Oh Devil,2017-06-24T17:21:00.133Z,0.896,0.526,-7.732,0.0578,0.12400,0.030700,0.1850,0.730,97.012,0.388889,0.016893,0.021301,0.021236,0.021301,0.018837,0.333333,0.018837,0.000000,0.416667,Saturday,June,24,Afternoon
1071,3dKGK0ALdWHldGJEXpwt3K,The Blasters,Dark Night,2017-06-24T23:21:52.199Z,0.597,0.730,-7.480,0.0346,0.00895,0.017000,0.1220,0.507,129.746,0.049412,0.184444,0.171429,0.077424,0.173810,0.350000,0.061657,0.212500,0.087202,0.287202,Saturday,June,24,Evening
1635,5piJiL6aRhvrKBa9YuEfS9,The Doors,Peace Frog,2017-06-24T22:59:53.842Z,0.641,0.725,-7.376,0.0390,0.16000,0.003210,0.6480,0.808,112.529,0.036106,0.129547,0.123810,0.053725,0.058631,0.065255,0.036106,0.067778,0.053841,0.176138,Saturday,June,24,Evening
1682,4v3zlxjqvlcFcWlIIOaEk9,Darondo,True,2017-06-24T19:47:11.287Z,0.861,0.483,-9.007,0.0311,0.31000,0.000101,0.0710,0.881,94.682,0.012203,0.037302,0.183333,0.040319,0.200000,0.094444,0.014298,0.094444,0.059167,0.036917,Saturday,June,24,Evening
1697,5lGAa7HUT1JmNXC1uXcv6x,Electric Wire Hustle,They Don't Want,2017-06-24T18:18:54.053Z,0.580,0.441,-9.863,0.1210,0.06550,0.080800,0.1720,0.781,117.945,0.056362,0.000000,0.000000,0.291667,0.000000,0.000000,0.263889,0.000000,0.000000,0.086409,Saturday,June,24,Evening
1730,1MvH3FdvO17S2eQ4IeNRCo,One Self,Bluebird,2017-06-24T18:26:31.791Z,0.780,0.797,-5.964,0.3070,0.12900,0.000000,0.0929,0.648,86.235,0.050275,0.010004,0.021840,0.105813,0.033719,0.017596,0.091475,0.011370,0.092063,0.078057,Saturday,June,24,Evening
1778,17wJbkfSi42AQP6h0zId0I,Ann Peebles,"Run, Run, Run",2017-06-24T18:40:15.121Z,0.757,0.343,-13.691,0.0540,0.24100,0.098100,0.0638,0.694,160.294,0.101852,0.093750,0.166667,1.000000,0.166667,0.090278,0.187500,0.093750,0.166667,0.250000,Saturday,June,24,Evening


In [ ]:
#Re-index june_data
june_data = reindex_df(june_data) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
june_data

,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Day Of Week,Month,Day of Month,Time Of Day
0,1SZThDcsBdExLGgsGF4jwj,Michael Franti & Spearhead,People In Tha Middle,2017-06-24T18:57:25.899Z,0.739,0.440,-9.582,0.2400,0.32200,0.009570,0.1820,0.893,171.118,0.040275,0.026115,0.018587,0.111111,0.043264,0.043264,0.038182,0.083333,0.375000,0.059799,Saturday,June,24,Evening
1,2P5mABk27DkOT8Gi2NbTpO,Madlib,Slim's Return,2017-06-24T18:22:51.015Z,0.508,0.668,-7.281,0.2210,0.04120,0.032700,0.3040,0.548,91.590,0.375000,0.200000,0.200000,0.250000,0.200000,0.200000,0.250000,0.000000,0.200000,0.375000,Saturday,June,24,Evening
2,6mz1fBdKATx6qP4oP1I65G,Ginuwine,Pony,2017-06-09T08:03:57.958Z,0.749,0.605,-9.359,0.0860,0.00186,0.038100,0.1150,0.966,142.024,0.035914,0.112500,0.266667,0.000000,0.200000,0.266667,0.045882,0.205556,1.000000,0.555556,Friday,June,9,Morning
3,3qQWGXvNwIOrskKOK9u9A0,Diana Ross,It's My House,2017-06-26T13:29:33.752Z,0.762,0.670,-5.862,0.0717,0.29000,0.000006,0.0783,0.870,99.473,0.036113,0.092500,0.100000,0.116667,0.100000,0.058750,0.050573,0.070972,0.150000,0.131250,Monday,June,26,Afternoon
4,7c3gg0yaitZpzx3EbuD2ch,Fatboy Slim,"Build It Up, Tear It Down",2017-06-21T09:12:08.790Z,0.615,0.863,-10.949,0.1080,0.00762,0.118000,0.2550,0.516,144.611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Wednesday,June,21,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,6ixFcE600rVVI1RJtcsmdq,Degrees Of Motion,Do You Want It Right Now - King Street Mix,2017-06-04T17:25:32.482Z,0.575,0.930,-5.873,0.0364,0.00189,0.000552,0.3290,0.550,117.989,0.053584,0.064444,0.133333,0.000000,0.133333,0.048704,0.053584,0.064444,0.107639,0.154514,Sunday,June,4,Afternoon
216,3wDyYQDkBSoqofcQ3eZJ9w,George McCrae,I Get Lifted,2017-06-13T21:32:18.840Z,0.796,0.523,-11.356,0.0432,0.56600,0.275000,0.0987,0.928,100.172,0.137500,0.000000,0.000000,0.000000,0.000000,1.000000,0.137500,0.000000,0.000000,0.333333,Tuesday,June,13,Evening
217,3dQqL6Z0ha4TkBGWiCeywt,FPI Project,Going Back To My Roots - Original Vocal Mix,2017-06-09T18:50:50.486Z,0.615,0.951,-6.159,0.0492,0.19800,0.005470,0.2220,0.686,120.238,0.032252,0.152778,0.500000,0.050556,0.000000,0.057778,0.083333,0.333333,0.000000,0.083333,Friday,June,9,Evening
218,3rv2u0rfXCZrUoeposkCML,Mary Clark,Take Me I'm Yours,2017-06-19T18:20:37.845Z,0.673,0.739,-9.935,0.0415,0.01260,0.000839,0.0527,0.850,105.132,0.017263,0.333333,0.000000,0.133333,0.000000,0.000000,0.018545,0.333333,0.133333,0.027613,Monday,June,19,Evening


#Finding Top Emotions for June 29 

After some inspection, we see that June 29 is a good day to investigate and use as a prototype for our dashboard, as it corresponds to multiple songs, and songs in each timeframe (i.e. morning, afternoon, evening).

In [ ]:
#create data list for df_june_emotions
data_june29_emotions = []

In [ ]:
#Get subsets for June 29 
df_june29_morning = june_data.loc[(june_data['Day of Month'] == 29) & (june_data['Time Of Day'] == "Morning")]                                                
df_june29_afternoon = june_data.loc[(june_data['Day of Month'] == 29) & (june_data['Time Of Day'] == "Afternoon")]
df_june29_evening = june_data.loc[(june_data['Day of Month'] == 29) & (june_data['Time Of Day'] == "Evening")]


## June 29: Morning

In [ ]:
#Get top emotion labels for June 29 morning
df_june29_morning

,Track ID,Artist,Track,Time Played At,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Day Of Week,Month,Day of Month,Time Of Day
57,4YwNf1fLZt9IQhJeHNWfBi,J. Cole,Intro,2017-06-29T11:00:01.620Z,0.435,0.268,-15.164,0.0438,0.844,0.0632,0.107,0.119,78.802,0.416667,0.5,1.0,0.541667,0.0,0.0,0.25,0.5,0.0,0.25,Thursday,June,29,Morning


From inspection, we see that there is only one entry recorded for the morning of June 29. The top three emotions, in order, are :

1. **Anger** *(score 1.00)*
2. **Anticipation** *(score 0.54)*
3. **Sadness** *(score 0.500)*

In [ ]:
#Add June 29 Morning data to data_june_emotions
data_june29_emotions.append(["Thursday", "June", 29 , "Morning", "Anger", 1.0, "Anticipation", 0.54, "Sadness", 0.5  ])

##June 29: Afternoon

In [ ]:
#Get top emotion labels for June 29 afternoon 
df_june29_afternoon.shape

#Create dict with keys for all emotions 
emotions_dict = {}

#Populate dict with mean values of emotions 
emotions_dict[df_june29_afternoon["Anger"].mean()] = "Anger"
emotions_dict[df_june29_afternoon["Anticipation"].mean()] = "Anticipation"
emotions_dict[df_june29_afternoon["Disgust"].mean()] = "Disgust"
emotions_dict[df_june29_afternoon["Fear"].mean()] = "Fear"
emotions_dict[df_june29_afternoon["Joy"].mean()] = "Joy"
emotions_dict[df_june29_afternoon["Sadness"].mean()] = 'Sadness'
emotions_dict[df_june29_afternoon["Surprise"].mean()] = "Surprise"
emotions_dict[df_june29_afternoon["Trust"].mean()] = "Trust"


#There are 3 entries for June 29 afternoon so we obtain the average emotion scores of the 3 entries and sort them in descending order
june29_afternoon_topEmotions = sorted([df_june29_afternoon["Anger"].mean(),df_june29_afternoon["Anticipation"].mean(), df_june29_afternoon["Disgust"].mean(), df_june29_afternoon["Fear"].mean(), df_june29_afternoon["Joy"].mean(), df_june29_afternoon["Sadness"].mean(), df_june29_afternoon["Surprise"].mean(), df_june29_afternoon["Trust"].mean()], reverse=True)

#Find corresponding emotions to top 3 highest values
top1 = emotions_dict.get(june29_afternoon_topEmotions[0])
top2 = emotions_dict.get(june29_afternoon_topEmotions[1])
top3 = emotions_dict.get(june29_afternoon_topEmotions[2])

emotions_dict
print("1: ", top1, "score: ", round(june29_afternoon_topEmotions[0], 4), " , ", "2: ", top2, "score: ", round(june29_afternoon_topEmotions[1], 4), " , ", "3: ", top3, "score: ", round(june29_afternoon_topEmotions[2], 4) )

1:  Anticipation score:  0.0842  ,  2:  Surprise score:  0.0307  ,  3:  Joy score:  0.0305


The top 3 emotions for the afternoon of June 29 are the following: 

1. **Anticipation** *(score: 0.084)*
2. **Surprise** *(score: 0.031)*
3. **Joy** *(score: 0.030)*

In [ ]:
data_june29_emotions.append(["Thursday", "June", 29 , "Afternoon", top1, june29_afternoon_topEmotions[0], top2, june29_afternoon_topEmotions[1], top3, june29_afternoon_topEmotions[2]])

##June 29: Evening

In [ ]:
#Get top emotion labels for June 29 evening
df_june29_evening.shape

#Create dict with keys for all emotions 
emotions_dict = {}

#Populate dict with mean values of emotions 
emotions_dict[df_june29_evening["Anger"].mean()] = "Anger"
emotions_dict[df_june29_evening["Anticipation"].mean()] = "Anticipation"
emotions_dict[df_june29_evening["Disgust"].mean()] = "Disgust"
emotions_dict[df_june29_evening["Fear"].mean()] = "Fear"
emotions_dict[df_june29_evening["Joy"].mean()] = "Joy"
emotions_dict[df_june29_evening["Sadness"].mean()] = 'Sadness'
emotions_dict[df_june29_evening["Surprise"].mean()] = "Surprise"
emotions_dict[df_june29_evening["Trust"].mean()] = "Trust"


#There are 3 entries for June 29 evening so we obtain the average emotion scores of the 3 entries and sort them in descending order
june29_evening_topEmotions = sorted([df_june29_evening["Anger"].mean(),df_june29_evening["Anticipation"].mean(), df_june29_evening["Disgust"].mean(), df_june29_evening["Fear"].mean(), df_june29_evening["Joy"].mean(), df_june29_evening["Sadness"].mean(), df_june29_evening["Surprise"].mean(), df_june29_evening["Trust"].mean()], reverse=True)

#Find corresponding emotions to top 3 highest values
top1 = emotions_dict.get(june29_evening_topEmotions[0])
top2 = emotions_dict.get(june29_evening_topEmotions[1])
top3 = emotions_dict.get(june29_evening_topEmotions[2])

emotions_dict
print("1: ", top1, "score: ", round(june29_evening_topEmotions[0], 4), " , ", "2: ", top2, "score: ", round(june29_evening_topEmotions[1], 4), " , ", "3: ", top3, "score: ", round(june29_evening_topEmotions[2],4) )

1:  Anticipation score:  0.4528  ,  2:  Joy score:  0.3039  ,  3:  Anger score:  0.1945


The top 3 emotions for the evening of June 29 are the following: 

1. **Anticipation** *(score: 0.45)*
2. **Joy** *(score: 0.30)*
3. **Anger** *(score: 0.19)*

In [ ]:
data_june29_emotions.append(["Thursday", "June", 29 , "Evening", top1, june29_evening_topEmotions[0], top2, june29_evening_topEmotions[1], top3, june29_evening_topEmotions[2]])

##June 29: Full day 

In [ ]:
#Get top emotion labels for June 29 
df_june29 = df_june29_morning
df_june29 = df_june29.append(df_june29_afternoon)
df_june29 = df_june29.append(df_june29_evening)

#Create dict with keys for all emotions 
emotions_dict = {}

#Populate dict with mean values of emotions 
emotions_dict[df_june29["Anger"].mean()] = "Anger"
emotions_dict[df_june29["Anticipation"].mean()] = "Anticipation"
emotions_dict[df_june29["Disgust"].mean()] = "Disgust"
emotions_dict[df_june29["Fear"].mean()] = "Fear"
emotions_dict[df_june29["Joy"].mean()] = "Joy"
emotions_dict[df_june29["Sadness"].mean()] = 'Sadness'
emotions_dict[df_june29["Surprise"].mean()] = "Surprise"
emotions_dict[df_june29["Trust"].mean()] = "Trust"


#There are 3 entries for June 29 evening so we obtain the average emotion scores of the 3 entries and sort them in descending order
june29_topEmotions = sorted([df_june29["Anger"].mean(),df_june29["Anticipation"].mean(), df_june29["Disgust"].mean(), df_june29["Fear"].mean(), df_june29["Joy"].mean(), df_june29["Sadness"].mean(), df_june29["Surprise"].mean(), df_june29["Trust"].mean()], reverse=True)

#Find corresponding emotions to top 3 highest values
top1 = emotions_dict.get(june29_topEmotions[0])
top2 = emotions_dict.get(june29_topEmotions[1])
top3 = emotions_dict.get(june29_topEmotions[2])

emotions_dict
print("1: ", top1, "score: ", june29_topEmotions[0], " , ", "2: ", top2, "score: ", june29_topEmotions[1], " , ", "3: ", top3, "score: ", june29_topEmotions[2] )

1:  Anticipation score:  0.3074984686004031  ,  2:  Anger score:  0.2293303020282187  ,  3:  Joy score:  0.17903572215923408


The top 3 emotions for the evening of June 29 are the following: 

1. **Anticipation** *(score: 0.31)*
2. **Anger** *(score: 0.23)*
3. **Joy** *(score: 0.18)*

In [ ]:
data_june29_emotions.append(["Thursday", "June", 29 , "Day", top1, june29_topEmotions[0], top2, june29_topEmotions[1], top3, june29_topEmotions[2]])

In [ ]:
df_june_emotions = pd.DataFrame(data_june29_emotions, columns= ['Day Of Week', 'Month', 'Day of Month', 'Time of Day', 'Emotion 1', 'Emotion 1 score','Emotion 2', 'Emotion 2 score', 'Emotion 3', 'Emotion 3 score' ])
df_june_emotions

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
0,Thursday,June,29,Morning,Anger,1.000000,Anticipation,0.540000,Sadness,0.500000
1,Thursday,June,29,Afternoon,Anticipation,0.084163,Surprise,0.030688,Joy,0.030492
2,Thursday,June,29,Evening,Anticipation,0.452778,Joy,0.303925,Anger,0.194491
3,Thursday,June,29,Day,Anticipation,0.307498,Anger,0.229330,Joy,0.179036


In [ ]:
#fn that returns the top 3 emotions from the input dataframe 
# @params:
    #df: dataframe
    # timefream: str

def get_top_emotions(df, timeframe):
  #Create dict with keys for all emotions 
  emotions_dict = {}

  #Populate dict with mean values of emotions 
  emotions_dict[df["Anger"].mean()] = "Anger"
  emotions_dict[df["Anticipation"].mean()] = "Anticipation"
  emotions_dict[df["Disgust"].mean()] = "Disgust"
  emotions_dict[df["Fear"].mean()] = "Fear"
  emotions_dict[df["Joy"].mean()] = "Joy"
  emotions_dict[df["Sadness"].mean()] = 'Sadness'
  emotions_dict[df["Surprise"].mean()] = "Surprise"
  emotions_dict[df["Trust"].mean()] = "Trust"

  #Sort avg emotion scores in descending order
  topEmotions = sorted([df["Anger"].mean(),df["Anticipation"].mean(), df["Disgust"].mean(), df["Fear"].mean(), df["Joy"].mean(), df["Sadness"].mean(), df["Surprise"].mean(), df["Trust"].mean()], reverse=True)

  #Find corresponding emotions to top 3 highest scores
  top1 = emotions_dict.get(topEmotions[0])
  top2 = emotions_dict.get(topEmotions[1])
  top3 = emotions_dict.get(topEmotions[2])

  if topEmotions[0]==0:
    top1 = None
  if topEmotions[1] ==0:
    top2 = None
  if topEmotions[2]==0:
    top3 =None

  return [timeframe, top1, topEmotions[0], top2, topEmotions[1], top3, topEmotions[2]]






In [ ]:
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

#fn that returns a list of all top_emotions_lists for all days, with specified timeframes 
# @param:
    #df_month : dataframe corresponding to all tracks listened to in a month 
def get_month_emotions(df_month):
  
  #init
  data_month_emotions = []
  morning_res = ['Morning', None, None, None, None, None, None]
  afternoon_res = ['Afternoon', None, None, None, None, None, None]
  evening_res = ['Evening', None, None, None, None, None, None]
  day_res = ['Day', None, None, None, None, None, None]

  #reindex month
  df_month = reindex_df(df_month)

  #extract info about param 
  str_month = df_month["Month"][0]

  # #create list of days in month
  days_in_month = []

  #extract list of days in month
  for i in range(0, df_month.shape[0]):
    #add day in month we have data for
    if df_month.loc[i,'Day of Month'] not in days_in_month:
      days_in_month.append(df_month.loc[i, 'Day of Month'])

  #order list of days in month
  days_in_month = sorted(days_in_month)

  for day in days_in_month:
    #extract day df
    df_day = get_df_DAY(df_month, str_month, day)

    #reindex day 
    df_day = reindex_df(df_day)

    #extract info about day 
    str_DoW = df_day['Day Of Week'][0]
    str_DoM = df_day['Day of Month'][0]


    #extract timeframe df
    df_morning = df_day.loc[df_day['Time Of Day'] == "Morning"]
    df_afternoon = df_day.loc[df_day['Time Of Day'] == "Afternoon"]
    df_evening = df_day.loc[df_day['Time Of Day'] == "Evening"]

    #base
    base = [str_DoW, str_month, str_DoM]

    #get top 3 emotions from each timeframe and the whole day 
    #if timeframe data exists for day, get emotions for timeframe
    if df_morning.shape[0] > 0 :
      morning_res = get_top_emotions(df_morning, "Morning")
    if df_afternoon.shape[0] > 0 :
      afternoon_res = get_top_emotions(df_afternoon, "Afternoon")
    if df_evening.shape[0] > 0 :
      evening_res = get_top_emotions(df_evening, "Evening")

    morning_res = base + morning_res
    afternoon_res = base + afternoon_res
    evening_res = base + evening_res


    #get emotions for the whole day 
    day_res = get_top_emotions(df_day, "Day")
    day_res = base + day_res

    #append to data list
    data_month_emotions.append(morning_res)
    data_month_emotions.append(afternoon_res)
    data_month_emotions.append(evening_res)
    data_month_emotions.append(day_res)
  
  return data_month_emotions 


In [ ]:
#fn that returns df_month, given a specified month,
# @params:
    #df: overall dataframe of songs 
    #month: str 

def get_df_MONTH(df, month):
  try:
    return df[df["Month"] == month]
  except:
    print("month does not exist in data")
    return


In [ ]:
#fn that returns a specified df_day, given a specifed month, day 
# @params:
    #df: overall dataframe of songs 
    #month: str 
    #day: int 
def get_df_DAY(df, month, day):
  #if day exists in df
  try:
    return df[(df["Month"] == month) & (df["Day of Month"] == day)]
  #day does not exist in df
  except:
    assert("no data for specified day")
    return 


In [ ]:
#fn that extracts overall timeframe emotions, given a df that shows emotions across all timeframes for every day of the month 
# @params:
    #df_month: dataframe that shows emotions across all time_frames 
    #timeframe: str 

def get_timeframe_emotions(df, timeframe):
  try:
    return df[df["Time of Day"] == timeframe]
  except:
    assert("no data for timeframe")
    return 

##Finding Top Emotions for all days in June

In [ ]:
#Overall results for June 

#Get June data from overall df 
june_data = get_df_MONTH(df_dashboard, "June")

#drop rows with NaNs for timeframes
june_data.dropna(inplace = True)

In [ ]:
#Get June emotions - emotions across all timeframes, for all days in June
df_june_emotions = pd.DataFrame(data = get_month_emotions(june_data))

#drop excess cols 
df_june_emotions.drop(df_june_emotions.iloc[:, 10:28], inplace = True, axis = 1)

#rename cols
df_june_emotions.columns = ['Day Of Week', 'Month', 'Day of Month', 'Time of Day', 'Emotion 1', 'Emotion 1 score','Emotion 2', 'Emotion 2 score', 'Emotion 3', 'Emotion 3 score']

#convert None to NaN
df_june_emotions.fillna(value=pd.np.nan, inplace=True)

#drop rows with NaN
df_june_emotions.dropna(axis=0, how='any',subset=['Emotion 1','Emotion 1 score', 'Emotion 2', 'Emotion 2 score', 'Emotion 3', 'Emotion 3 score'], inplace=True)

df_june_emotions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  # This is added back by InteractiveShellApp.init_path()


,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
1,Friday,June,2,Afternoon,Surprise,0.277778,Anticipation,0.217593,Anger,0.151112
2,Friday,June,2,Evening,Anticipation,0.184922,Surprise,0.182003,Anger,0.124319
3,Friday,June,2,Day,Surprise,0.205947,Anticipation,0.193089,Anger,0.131017
4,Sunday,June,4,Morning,Anticipation,0.461304,Trust,0.212462,Surprise,0.148148
5,Sunday,June,4,Afternoon,Anger,0.150931,Trust,0.14721,Sadness,0.134977
...,...,...,...,...,...,...,...,...,...,...
79,Wednesday,June,28,Day,Trust,0.107520,Anger,0.104852,Joy,0.0895723
80,Thursday,June,29,Morning,Anger,1.000000,Anticipation,0.541667,Sadness,0.5
81,Thursday,June,29,Afternoon,Anticipation,0.084163,Surprise,0.0306878,Joy,0.0304916
82,Thursday,June,29,Evening,Anticipation,0.452778,Joy,0.303925,Anger,0.194491


In [ ]:
#Overall daily top emotions for all days in June
df_june_emotions_alldays = get_timeframe_emotions(df_june_emotions, "Day")

df_june_emotions_alldays

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
3,Friday,June,2,Day,Surprise,0.205947,Anticipation,0.193089,Anger,0.131017
7,Sunday,June,4,Day,Anticipation,0.137670,Trust,0.121311,Surprise,0.0973013
11,Monday,June,5,Day,Sadness,0.192270,Surprise,0.156904,Disgust,0.154984
15,Wednesday,June,7,Day,Anticipation,0.151716,Surprise,0.122446,Trust,0.117586
19,Thursday,June,8,Day,Surprise,0.186129,Trust,0.106558,Fear,0.0903456
23,Friday,June,9,Day,Surprise,0.222132,Anger,0.124372,Fear,0.106224
27,Saturday,June,10,Day,Anticipation,0.270833,Disgust,0.266667,Trust,0.1875
31,Monday,June,12,Day,Surprise,0.195112,Fear,0.19145,Anger,0.113928
35,Tuesday,June,13,Day,Fear,0.302950,Trust,0.117485,Disgust,0.116258
39,Wednesday,June,14,Day,Surprise,0.237500,Disgust,0.213889,Sadness,0.191113


##Finding Top Emotions across all mornings in June

In [ ]:
#Avg morning emotion across all days in June
df_june_emotions_allmornings = get_timeframe_emotions(df_june_emotions, "Morning")

df_june_emotions_allmornings

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
4,Sunday,June,4,Morning,Anticipation,0.461304,Trust,0.212462,Surprise,0.148148
8,Monday,June,5,Morning,Sadness,0.192270,Surprise,0.156904,Disgust,0.154984
12,Wednesday,June,7,Morning,Anticipation,0.151716,Surprise,0.122446,Trust,0.117586
16,Thursday,June,8,Morning,Surprise,0.121037,Sadness,0.047619,Sadness,0.047619
20,Friday,June,9,Morning,Surprise,0.322174,Trust,0.118011,Disgust,0.117344
24,Saturday,June,10,Morning,Anticipation,0.270833,Disgust,0.266667,Trust,0.1875
28,Monday,June,12,Morning,Surprise,0.186492,Fear,0.163966,Anger,0.108347
36,Wednesday,June,14,Morning,Surprise,0.237500,Disgust,0.213889,Sadness,0.191113
40,Saturday,June,17,Morning,Surprise,0.147950,Trust,0.121271,Disgust,0.0747014
44,Monday,June,19,Morning,Anticipation,0.237783,Sadness,0.183514,Trust,0.167117


##Finding Top Emotions across all afternoons in June

In [ ]:
#Avg afternoon emotion across all days in June 
df_june_emotions_allafternoons = get_timeframe_emotions(df_june_emotions, "Afternoon")

df_june_emotions_allafternoons

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
1,Friday,June,2,Afternoon,Surprise,0.277778,Anticipation,0.217593,Anger,0.151112
5,Sunday,June,4,Afternoon,Anger,0.150931,Trust,0.14721,Sadness,0.134977
17,Thursday,June,8,Afternoon,Surprise,0.218676,Trust,0.145037,Fear,0.111709
29,Monday,June,12,Afternoon,Fear,0.242982,Surprise,0.211273,Joy,0.175127
57,Friday,June,23,Afternoon,Anger,0.500000,Joy,0.333333,Fear,0.222222
61,Saturday,June,24,Afternoon,Trust,0.208333,Sadness,0.176085,Sadness,0.176085
69,Monday,June,26,Afternoon,Disgust,0.105000,Anticipation,0.093725,Surprise,0.0910139
73,Tuesday,June,27,Afternoon,Anticipation,0.583333,Fear,0.3125,Disgust,0.166667
81,Thursday,June,29,Afternoon,Anticipation,0.084163,Surprise,0.0306878,Joy,0.0304916


##Finding Top Emotions across all evenings in June

In [ ]:
#Avg evening emotion across all days in June 
df_june_emotions_allevenings = get_timeframe_emotions(df_june_emotions, "Evening")

df_june_emotions_allevenings

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
2,Friday,June,2,Evening,Anticipation,0.184922,Surprise,0.182003,Anger,0.124319
6,Sunday,June,4,Evening,Anticipation,0.070522,Trust,0.067706,Surprise,0.0594823
22,Friday,June,9,Evening,Surprise,0.152102,Anger,0.139887,Fear,0.101107
34,Tuesday,June,13,Evening,Fear,0.302950,Trust,0.117485,Disgust,0.116258
46,Monday,June,19,Evening,Anticipation,0.153357,Sadness,0.12407,Trust,0.0776783
54,Wednesday,June,21,Evening,Sadness,0.179304,Anger,0.17463,Surprise,0.117376
58,Friday,June,23,Evening,Surprise,0.171033,Disgust,0.132953,Anger,0.094666
62,Saturday,June,24,Evening,Anticipation,0.158809,Fear,0.154163,Trust,0.146493
66,Sunday,June,25,Evening,Surprise,0.270833,Disgust,0.104167,Anticipation,0.0708333
70,Monday,June,26,Evening,Trust,0.203651,Anticipation,0.191429,Anger,0.147619


##Finding Top Emotions for a specific day

In [ ]:
#For June 4, get emotions across all timeframes
df_june4 = get_df_DAY(df_june_emotions, "June", 4)

#For June 3, get emotions across all timeframes 
df_june3 = get_df_DAY(df_june_emotions, "June", 3)


In [ ]:
df_june4

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
4,Sunday,June,4,Morning,Anticipation,0.461304,Trust,0.212462,Surprise,0.148148
5,Sunday,June,4,Afternoon,Anger,0.150931,Trust,0.14721,Sadness,0.134977
6,Sunday,June,4,Evening,Anticipation,0.070522,Trust,0.067706,Surprise,0.0594823
7,Sunday,June,4,Day,Anticipation,0.137670,Trust,0.121311,Surprise,0.0973013


In [ ]:
df_june3

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score


##Finding Top Emotions for a specific Time of Day

In [ ]:
#For June 4, get emotions for morning 
df_june4_morning = get_timeframe_emotions(df_june4, "Morning")

df_june4_morning

,Day Of Week,Month,Day of Month,Time of Day,Emotion 1,Emotion 1 score,Emotion 2,Emotion 2 score,Emotion 3,Emotion 3 score
4,Sunday,June,4,Morning,Anticipation,0.461304,Trust,0.212462,Surprise,0.148148


# Finding Top Emotion of Time of Day:

In [ ]:
morning_data = df_model[df_model["Time Of Day"] == "Morning"]
afternoon_data = df_model[df_model["Time Of Day"] == "Afternoon"]
evening_data = df_model[df_model["Time Of Day"] == "Evening"]
all_time_data = [morning_data, afternoon_data, evening_data]
emotions = ["Anger","Anticipation",	"Disgust",	"Fear",	"Joy", "Sadness",	"Surprise",	"Trust"]
time_title = ["Morning", "Afternoon", "Evening"]

#finding the overall top emotion for each month
emotion_value = []
overall_emotion = ""
loc = 0
for month in all_time_data:
  for emotion in emotions:
    emotion_value.append(month[emotion].sum())
  emotion_loc = emotion_value.index(max(emotion_value))
  print(time_title[loc]+" top emotion:", emotions[emotion_loc])
  loc+=1
  emotion_value = []

Morning top emotion: Anticipation
Afternoon top emotion: Anticipation
Evening top emotion: Anticipation


In [ ]:
april_data = df_model[df_model["Month"] == "April"]
april_monday_data= april_data[april_data["Day Of Week"] == "Monday"]
april_tuesday_data= april_data[april_data["Day Of Week"] == "Tuesday"]
april_wednesday_data= april_data[april_data["Day Of Week"] == "Wednesday"]
april_thursday_data= april_data[april_data["Day Of Week"] == "Thursday"]
april_friday_data= april_data[april_data["Day Of Week"] == "Friday"]
april_saturday_data= april_data[april_data["Day Of Week"] == "Saturday"]
april_sunday_data= april_data[april_data["Day Of Week"] == "Sunday"]
april_week_data = [april_monday_data, april_tuesday_data, april_wednesday_data, april_thursday_data, april_friday_data, april_saturday_data, april_sunday_data]
emotions = ["Anger","Anticipation",	"Disgust",	"Fear",	"Joy", "Sadness",	"Surprise",	"Trust"]
day_title = ["April Monday", "April Tuesday", "April Wednesday", "April Thursday", "April Friday", "April Saturday", "April Sunday"]

#finding the overall top emotion for each month
emotion_value = []
overall_emotion = ""
loc = 0
for day in april_week_data:
  for emotion in emotions:
    emotion_value.append(day[emotion].sum())
  emotion_loc = emotion_value.index(max(emotion_value))
  print(day_title[loc]+" top emotion:", emotions[emotion_loc])
  loc+=1
  emotion_value = []

April Monday top emotion: Anticipation
April Tuesday top emotion: Anticipation
April Wednesday top emotion: Anticipation
April Thursday top emotion: Anticipation
April Friday top emotion: Anticipation
April Saturday top emotion: Anticipation
April Sunday top emotion: Sadness


In [ ]:
may_data = df_model[df_model["Month"] == "May"]
may_monday_data= may_data[may_data["Day Of Week"] == "Monday"]
may_tuesday_data= may_data[may_data["Day Of Week"] == "Tuesday"]
may_wednesday_data= may_data[may_data["Day Of Week"] == "Wednesday"]
may_thursday_data= may_data[may_data["Day Of Week"] == "Thursday"]
may_friday_data= may_data[may_data["Day Of Week"] == "Friday"]
may_saturday_data= may_data[may_data["Day Of Week"] == "Saturday"]
may_sunday_data= may_data[may_data["Day Of Week"] == "Sunday"]
may_week_data = [may_monday_data, may_tuesday_data, may_wednesday_data, may_thursday_data, may_friday_data, may_saturday_data, may_sunday_data]
emotions = ["Anger","Anticipation",	"Disgust",	"Fear",	"Joy", "Sadness",	"Surprise",	"Trust"]
day_title = ["may Monday", "may Tuesday", "may Wednesday", "may Thursday", "may Friday", "may Saturday", "may Sunday"]

#finding the overall top emotion for each month
emotion_value = []
overall_emotion = ""
loc = 0
for day in may_week_data:
  for emotion in emotions:
    emotion_value.append(day[emotion].sum())
  emotion_loc = emotion_value.index(max(emotion_value))
  print(day_title[loc]+" top emotion:", emotions[emotion_loc])
  loc+=1
  emotion_value = []

may Monday top emotion: Anticipation
may Tuesday top emotion: Anticipation
may Wednesday top emotion: Anticipation
may Thursday top emotion: Surprise
may Friday top emotion: Anticipation
may Saturday top emotion: Anticipation
may Sunday top emotion: Trust


In [ ]:
june_data = df_model[df_model["Month"] == "June"]
june_monday_data= june_data[june_data["Day Of Week"] == "Monday"]
june_tuesday_data= june_data[june_data["Day Of Week"] == "Tuesday"]
june_wednesday_data= june_data[june_data["Day Of Week"] == "Wednesday"]
june_thursday_data= june_data[june_data["Day Of Week"] == "Thursday"]
june_friday_data= june_data[june_data["Day Of Week"] == "Friday"]
june_saturday_data= june_data[june_data["Day Of Week"] == "Saturday"]
june_sunday_data= june_data[june_data["Day Of Week"] == "Sunday"]
june_week_data = [june_monday_data, june_tuesday_data, june_wednesday_data, june_thursday_data, june_friday_data, june_saturday_data, june_sunday_data]
emotions = ["Anger","Anticipation",	"Disgust",	"Fear",	"Joy", "Sadness",	"Surprise",	"Trust"]
day_title = ["june Monday", "june Tuesday", "june Wednesday", "june Thursday", "june Friday", "june Saturday", "june Sunday"]

#finding the overall top emotion for each month
emotion_value = []
overall_emotion = ""
loc = 0
for day in june_week_data:
  for emotion in emotions:
    emotion_value.append(day[emotion].sum())
  emotion_loc = emotion_value.index(max(emotion_value))
  print(day_title[loc]+" top emotion:", emotions[emotion_loc])
  loc+=1
  emotion_value = []

june Monday top emotion: Anticipation
june Tuesday top emotion: Surprise
june Wednesday top emotion: Anger
june Thursday top emotion: Anticipation
june Friday top emotion: Surprise
june Saturday top emotion: Trust
june Sunday top emotion: Anticipation


In [ ]:
def day_mood_finder(df_model):
  morning_data = df_model[df_model["Time Of Day"] == "Morning"]
  afternoon_data = df_model[df_model["Time Of Day"] == "Afternoon"]
  evening_data = df_model[df_model["Time Of Day"] == "Evening"]
  all_time_data = [morning_data, afternoon_data, evening_data]
  emotions = ["Anger","Anticipation",	"Disgust",	"Fear",	"Joy", "Sadness",	"Surprise",	"Trust"]
  time_title = ["Morning", "Afternoon", "Evening"]

  #finding the overall top emotion for each month
  emotion_value = []
  overall_emotion = ""
  loc = 0
  for month in all_time_data:
    for emotion in emotions:
      emotion_value.append(month[emotion].sum())
    emotion_loc = emotion_value.index(max(emotion_value))
    print(time_title[loc]+" top emotion:", emotions[emotion_loc])
    loc+=1
    emotion_value = []

april_data = df_model[df_model["Month"] == "April"]
april_monday_data= april_data[april_data["Day Of Week"] == "Monday"]
april_tuesday_data= april_data[april_data["Day Of Week"] == "Tuesday"]
april_wednesday_data= april_data[april_data["Day Of Week"] == "Wednesday"]
april_thursday_data= april_data[april_data["Day Of Week"] == "Thursday"]
april_friday_data= april_data[april_data["Day Of Week"] == "Friday"]
april_saturday_data= april_data[april_data["Day Of Week"] == "Saturday"]
april_sunday_data= april_data[april_data["Day Of Week"] == "Sunday"]
april_week_data = [april_monday_data, april_tuesday_data, april_wednesday_data, april_thursday_data, april_friday_data, april_saturday_data, april_sunday_data]
day_title = ["April Monday:", "April Tuesday:", "April Wednesday:", "April Thursday:", "April Friday:", "April Saturday:", "April Sunday:"]
loc = 0

for april_days in april_week_data:
  print(day_title[loc])
  day_mood_finder(april_days)
  loc+=1
  print()


may_data = df_model[df_model["Month"] == "May"]
may_monday_data= may_data[may_data["Day Of Week"] == "Monday"]
may_tuesday_data= may_data[may_data["Day Of Week"] == "Tuesday"]
may_wednesday_data= may_data[may_data["Day Of Week"] == "Wednesday"]
may_thursday_data= may_data[may_data["Day Of Week"] == "Thursday"]
may_friday_data= may_data[may_data["Day Of Week"] == "Friday"]
may_saturday_data= may_data[may_data["Day Of Week"] == "Saturday"]
may_sunday_data= may_data[may_data["Day Of Week"] == "Sunday"]
may_week_data = [may_monday_data, may_tuesday_data, may_wednesday_data, may_thursday_data, may_friday_data, may_saturday_data, may_sunday_data]
day_title = ["May Monday:", "May Tuesday:", "May Wednesday:", "May Thursday:", "May Friday:", "May Saturday:", "MayMay Sunday:"]
loc = 0

for may_days in may_week_data:
  print(day_title[loc])
  day_mood_finder(may_days)
  loc+=1
  print()


june_data = df_model[df_model["Month"] == "June"]
june_monday_data= june_data[june_data["Day Of Week"] == "Monday"]
june_tuesday_data= june_data[june_data["Day Of Week"] == "Tuesday"]
june_wednesday_data= june_data[june_data["Day Of Week"] == "Wednesday"]
june_thursday_data= june_data[june_data["Day Of Week"] == "Thursday"]
june_friday_data= june_data[june_data["Day Of Week"] == "Friday"]
june_saturday_data= june_data[june_data["Day Of Week"] == "Saturday"]
june_sunday_data= june_data[june_data["Day Of Week"] == "Sunday"]
june_week_data = [june_monday_data, june_tuesday_data, june_wednesday_data, june_thursday_data, june_friday_data, june_saturday_data, june_sunday_data]
day_title = ["June Monday:", "June Tuesday:", "June Wednesday:", "June Thursday:", "June Friday:", "June Saturday:", "June Sunday:"]
loc = 0

for june_days in june_week_data:
  print(day_title[loc])
  day_mood_finder(june_days)
  loc+=1
  print()


April Monday:
Morning top emotion: Surprise
Afternoon top emotion: Fear
Evening top emotion: Surprise

April Tuesday:
Morning top emotion: Sadness
Afternoon top emotion: Sadness
Evening top emotion: Anticipation

April Wednesday:
Morning top emotion: Anticipation
Afternoon top emotion: Surprise
Evening top emotion: Fear

April Thursday:
Morning top emotion: Anticipation
Afternoon top emotion: Anticipation
Evening top emotion: Sadness

April Friday:
Morning top emotion: Trust
Afternoon top emotion: Anticipation
Evening top emotion: Anticipation

April Saturday:
Morning top emotion: Anger
Afternoon top emotion: Anticipation
Evening top emotion: Anticipation

April Sunday:
Morning top emotion: Anger
Afternoon top emotion: Disgust
Evening top emotion: Sadness

May Monday:
Morning top emotion: Anticipation
Afternoon top emotion: Anticipation
Evening top emotion: Anticipation

May Tuesday:
Morning top emotion: Sadness
Afternoon top emotion: Anger
Evening top emotion: Anticipation

May Wednes

# Finding the percentage of positive and negative songs for each week in June


We want to see the percentage of positive and negative songs the user listens to per week. 

We create two helper functions, one that looks at our dataframe and returns a list of weekNumbers from the "Time Played At" column. These weekNumbers correspond to the week of the year a track is played in. 

The next helper function gives us a subset of the original dataframe that corresponds to a weekNumber. This is so we can retrieve the songs that were played in a particular week of the year.


In [ ]:
#fn to retrieve list of weekNumbers in a dataset, in ascending order
#@param: df = dataframe
def get_weekNumbers(df):
  
  #Retrieve weekNumber for each date in df 
  df['Date'] = df['Time Played At'].str[0:10]
  df['Date'] = pd.to_datetime(df['Date'])
  df['Week Of Year'] = df['Date'].dt.week

  #Create list of weekNumbers
  list_of_weekNumbers = []

  #Populate list of weekNumbers
  for i in range(0, df.shape[0]):
    if df["Week Of Year"][i] not in list_of_weekNumbers:
      list_of_weekNumbers.append(df["Week Of Year"][i])
  return sorted(list_of_weekNumbers)


In [ ]:
#fn that retrieves subsets of original dataframe based on input weekNumber
#@param: df = dataframe

def get_weekDf(df, weekNumber):
  return df.loc[df['Week Of Year'] == weekNumber]
  

We now evaluate the month of June.

First, we look at the weekly subsets of data for the month of June to determine which songs were played in which week. 

For the songs belonging to a particular week, we create a txt file with the artistName, songName and trackID, which acts as an input for modified_lyricCollect.py. 

Our script, modified_lyricCollect.py, then outputs the overall percentage of positive and negative words found in the tracks from the input file.

In [ ]:
#Evaluating weekly dataframes from the month of June 

#Ensure dataframe is indexed correctly
june_data = reindex_df(june_data)

#Get the list of weeknumbers 
weekNums = get_weekNumbers(june_data)

for num in weekNums: 

  #retrieve df for weekNumber
  df_week = get_weekDf(june_data, num)

  #re-index df_week 
  df_week = reindex_df(df_week)

  #retrieve artist, song, and trackID for df_week 
  filename = "artistAndSongName_WEEK_" + str(num) + ".txt"
  print(filename)
  get_artist_and_song(df_week,filename)
  
  #download retrieved file 
  files.download(filename)



The following files were downloaded using the song data from June:

  - artistAndSongName_WEEK_22.txt
  - artistAndSongName_WEEK_23.txt
  - artistAndSongName_WEEK_24.txt
  - artistAndSongName_WEEK_25.txt
  - artistAndSongName_WEEK_26.txt

In [ ]:
#Create col_names for dataframe showing the weekNumbers of June and the percentage of  positive and percentage of negative words 
#col_names = ["Month", "Week", "% of Positive Words", "% of Negative Words"]

#Retrieve scores for each week of June 
root_path = "gdrive/MyDrive/Menon_Labs/code_and_data/"
June_week22_score = pd.read_csv(root_path + "artistAndSongName_WEEK_22_results.csv")
June_week23_score = pd.read_csv(root_path + "artistAndSongName_WEEK_23_results.csv")
June_week24_score = pd.read_csv(root_path + "artistAndSongName_WEEK_24_results.csv")
June_week25_score = pd.read_csv(root_path + "artistAndSongName_WEEK_25_results.csv")
June_week26_score = pd.read_csv(root_path + "artistAndSongName_WEEK_26_results.csv")

#Concatenate scores for all weeks
df_june_allweeks = pd.concat([June_week22_score, June_week23_score, June_week24_score, June_week25_score, June_week26_score ])




Above, you can see the percentage of positive words and negative words in the songs the user listened to each week.

We investigate the weekNumber for June 29 and the corresponding percentage of positive and negative words

In [ ]:
#Find weekNumber for June 29 2017 
date = "2017-06-29"
date = pd.to_datetime(date)
weeknum = date.week

#June 29 is part of week 26 
weeknum


26

We found out that June 29 is part of week 26. Therefore, a user viewing their dashboard on June 29 will be able to view the sentiment analysis of  positive and negative words from tracks they listened to on week 26.

In [ ]:
June_week26_score

,Input File,Percentage of Positive Words,Percentage of Negative Words
0,WEEK_26,52.84,47.16


#Finding the percentage positive and negative songs for the month of June

We use the mean of the percentage of positive words and the mean of the percentage of negative words found from every week in June to calculate the monthly average of percentage of positive and percentage of negative words for June

In [ ]:
june_avg_pos = df_june_allweeks['Percentage of Positive Words'].mean()
june_avg_neg = df_june_allweeks['Percentage of Negative Words'].mean()


print("Average percentage positive words for June: ", june_avg_pos)
print("Average percentage negative words for June: ", june_avg_neg)

Average percentage positive words for June:  50.452000000000005
Average percentage negative words for June:  49.547999999999995


We see that the average percentage of positive words from the tracks the user listened to in June is 50.45%, and the average percentage of negative words is 49.55% 

Therefore, on average, approximately half of the words from all the songs were positive and half of the words from all the songs were negative.

#Finding the most common themes of songs for the month of June

We then conducted LDA analysis on all the song lyrics for the month of June to see if any overall themes arised from the tracks.

This is to provide additional information to the users looking at their monthly Spotify insights.

In [ ]:
#Extract input file for whole month

#ensure df is reindexed
june_data = reindex_df(june_data)

#retrieve artist, song, and trackID for df_week 
filename = "lda_input.txt"
get_artist_and_song(june_data,filename)
  
#download retrieved file 
files.download(filename)



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#t

#topics = 6, words = 6 (BAD) [(0, '0.038*"niet" + 0.024*"Habiba" + 0.022*"zina" + 0.022*"booty" + 0.022*"dieven" + 0.019*"move"'), (1, '0.022*"saying" + 0.022*"love" + 0.021*"know" + 0.016*"Been" + 0.013*"every" + 0.013*"back"'), (2, '0.024*"bounce" + 0.011*"ever" + 0.010*"just" + 0.010*"back" + 0.009*"burn" + 0.009*"tonight"'), (3, '0.027*"yeah" + 0.017*"like" + 0.014*"know" + 0.014*"What" + 0.011*"doin" + 0.011*"feel"'), (4, '0.070*"druggy" + 0.050*"fucking" + 0.028*"heaven" + 0.022*"Slim" + 0.022*"Fatboy" + 0.020*"watch"'), (5, '0.019*"love" + 0.017*"right" + 0.015*"house" + 0.014*"Right" + 0.014*"baby" + 0.011*"know"')]
#topics = 4, words = 3  [(0, '0.013*"love" + 0.013*"yeah" + 0.012*"know"'), (1, '0.037*"druggy" + 0.012*"like" + 0.010*"yeah"'), (2, '0.026*"move" + 0.026*"right" + 0.024*"booty"'), (3, '0.022*"fucking" + 0.022*"niet" + 0.014*"Habiba"')]
#topics = 4, words = 4 [(0, '0.023*"fucking" + 0.016*"right" + 0.013*"heaven" + 0.013*"house"'), (1, '0.032*"druggy" + 0.022*"niet" + 0.014*"Habiba" + 0.013*"move"'), (2, '0.008*"love" + 0.008*"night" + 0.007*"Someday" + 0.006*"yeah"'), (3, '0.018*"yeah" + 0.017*"love" + 0.016*"baby" + 0.016*"What"')]
#topics = 4, words = 5 [(0, '0.049*"druggy" + 0.014*"yeah" + 0.013*"will" + 0.011*"Jack" + 0.011*"jack"'), (1, '0.018*"niet" + 0.012*"Habiba" + 0.011*"dieven" + 0.011*"zina" + 0.009*"right"'), (2, '0.021*"fucking" + 0.013*"love" + 0.013*"heaven" + 0.012*"house" + 0.012*"like"'), (3, '0.017*"love" + 0.016*"move" + 0.016*"yeah" + 0.016*"baby" + 0.015*"booty"')]
#topics = 3, words = 3 (BAD) [(0, '0.014*"love" + 0.012*"know" + 0.011*"like"'), (1, '0.013*"know" + 0.011*"right" + 0.010*"move"'), (2, '0.034*"druggy" + 0.025*"fucking" + 0.024*"niet"')]
#topics = 2, words = 3 (OK) [(0, '0.019*"fucking" + 0.018*"niet" + 0.011*"Habiba"'), (1, '0.013*"know" + 0.013*"druggy" + 0.012*"love"')]
#topics = 2, words = 4 (BAD) (0, '0.018*"druggy" + 0.013*"fucking" + 0.013*"niet" + 0.008*"Habiba"'), (1, '0.013*"know" + 0.012*"yeah" + 0.010*"like" + 0.009*"right"')]
#topics = 2, words = 5 (BAD) [(0, '0.013*"know" + 0.013*"druggy" + 0.012*"love" + 0.012*"yeah" + 0.010*"like"'), (1, '0.018*"niet" + 0.011*"Habiba" + 0.011*"dieven" + 0.011*"zina" + 0.011*"booty"')]
#topics = 1, words = 5 (BAD)  [(0, '0.009*"know" + 0.009*"druggy" + 0.008*"love" + 0.008*"yeah" + 0.007*"like"')]
#topics = 1, words = 3 (BAD) [(0, '0.009*"know" + 0.009*"druggy" + 0.008*"love"')]

To generate our LDA model, we tried the following parameters:

1. topics = 6, words per topic = 6
2. topics = 4, words per topic = 3
3. topics = 4, words per topic = 4
4. topics = 3, wods per topic = 3
5. topics = 2, words per topic = 3
6. topics = 2, words per topic = 4

In general, we found that a lower number of topics gave us more meaningful results. 

When selecting 4 topics and 4 words per topic, we believe we got the most meaningful results. The topics generated were the following:

- topic_0 = ("**f--king**, "right", "heaven", "house"")
- topic_1 = ("**druggy**", "niet", "Habiba", "move")
- topic_2 = ("**love**", "**night**", "Someday", "yeah)
-topic_3 = ("**yeah**", "**love**", "baby", "what")

The highest weighted word in each topic is in bold. The highest weighted words in topic_2 fits into the theme of "anticipation", and the one in topic_1 fits into the theme of "anger", which could explain why those were some of the prominent emotions we saw in the data for June.

However, our results were still lacking as the dataset composed of songs a user listened to in one month is very small (~220 entries). We beleive that if we conducted further LDA analysis on songs a user listens to over a longer time frame, we will get more meaningful results.



##**Predicting Mood using Audio and Lyrical Features**

After our feature engineering process of creating mood scores based on the lyrics of songs, we create a dataframe for our machine learning model. This is done by removing the duplicate tracks from the above dataframe, in order to reduce over-fitting.



In [ ]:
df_model = df_dashboard.drop_duplicates(subset=['Track ID'])

df_model